<a href="https://colab.research.google.com/github/visoutre/ai-notebooks/blob/main/Stable_Diffusion_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion Batch by [visoutre](https://www.reddit.com/user/visoutre)

Location for updates: [Stable Diffusion Batch.ipynb](https://colab.research.google.com/github/visoutre/ai-notebooks/blob/main/Stable_Diffusion_Batch.ipynb)

<b> v 2.4.0, last updated 9/05/22 </b>
<br>
- Expand to view info...


<u>**Credits:**</u>
- the setup work was done by [FutureArt](https://twitter.com/future__art) and [Pharmapsychotic](https://twitter.com/pharmapsychotic), credit goes to them for getting me started
- I added the bulk of features useful to concept art and img2img
- I used some public code to implement openCV facial recognition for face detailing
- The workflow for the bonus **HD Generation** scripts was inspired by Inspired by [u/tokidokiyuki](https://www.reddit.com/user/tokidokiyuki/)
[and their post on Reddit](https://www.reddit.com/r/StableDiffusion/comments/x45uk6/my_process_to_upscale_an_image_through_img2img/)
- [uses Real ESRGAN for upscaling](https://github.com/xinntao/Real-ESRGAN)

<u>**Terms of Use:**</u>
- This code is free to use however you want. You may use it, modify it and do anything with it without crediting me for the code (although it would be nice to get give me a mention if you use this code)
- Any images you create using this code fall under the terms set by [huggingface.co](https://huggingface.co/CompVis/stable-diffusion) , [Stability.Ai](https://stability.ai/) as well as Google Colab terms. You must use this technology ethically. I am not responsible for anything you create, nor do I own any of your creations
- I recommend only using input images you own the copywrite to, otherwise you may be liable to copywrite infringement 
- Any text prompts I share in code or elsewhere are free for you to use however you want, unrestricted and without credit to me. I believe prompts should be public and shared freely
- You may use any of your creations privately and commercially without credit
- Any images I create with this code are owned by me and you may not use any of my examples for commercial use, especially my original art. Respect me for sharing this code freely

<br> 

---

<u>**Key Features Include:**</u>
 - process a single image or multiple images, single prompts or multiple prompts
 - randomizing starting keyword in the prompts for variations on a theme
 - processing sequential guidance scales and image strengths on a seed for data
 - randomizing guidance scales and image strengths for exploration
 - processing a list of prompts with a corresponding image for consistency
 - processing a list of prompts with a random image for creativity
 - save prompt templates which can be quickly loaded
 - cycle through multiple prompt templates with multiple input images/prompts
 - the prompt cycling feature are like having your own art design team who can create different styles or follow a style!
 - sequence animation with ability to warp from after # of frames
 - face recognition to upscale the faces
 - HD upscaling by splitting the image into tiles

<u>**Upcoming Plans (todo list):**</u>
  - port locally to use with an RTX / better GUI (long term)


<br>

---
<u>**Changelog:**</u>

v 2.2.2, last updated 9/05/22:
- added Real ESRGAN to use by itself for upscaling images, or for use with face recognition
- got the face recognition to work and it will output files to Google Drive
- the final results of face recognition are not auto blended into the base image. How to do this cleanly is beyond me, so I just make use of masking the result with the base
- don't plan to add more code soon since there's a lot of features to work with

v 2.3.0, last updated 9/04/22:
- added the Bonus feature; HD Generation. Creates a generation for a 512x512 square across an input image. Inspired by [u/tokidokiyuki](https://www.reddit.com/user/tokidokiyuki/)
[and their post on Reddit](https://www.reddit.com/r/StableDiffusion/comments/x45uk6/my_process_to_upscale_an_image_through_img2img/)

v 2.2.2, last updated 9/02/22:
- cleaned up the documentation, cleaned up the sections and reordered some fields
- added aspect ratio templates

v 1 to 2:  updated between 8/27/22 and 9/01/22:
- experimented with core features to assist with concept art & design
- added interpolation, multiple input images, prompt templates, etc
- began exploring advanced features with openCV such as using facial recognition to clean up faces automatically

v 1.0.0, last updated 8/27/22:
- began the Stable Diffusion journey with FutureArt and Pharmapsychotic's collab file as a starting point
- my initial intention was to take their version that had multiple prompts and allow those prompts to correspond with their own unique input images

<br>

---

<u>**How to download the Stable Diffusion model:**</u>

- Visit https://huggingface.co/CompVis/stable-diffusion-v-1-4-original and agree to the terms and conditions.
- Click the **Files and versions** tab
- Click **stable-diffusion-v-1-4-original**
- Click the **download** link where it says *This file is stored with Git LFS . It is too big to display, but you can still download it.*
- If you have [Google Drive for desktop](https://www.google.com/drive/download/) (highly recommended), you can save it directly to your **AI/models** directory.
  - Otherwise, download it and re-upload it to your [Google Drive](https://drive.google.com) in the **AI/models** directory. (This is risky, as the upload may time out.)

# 1. <u> Setup (Run All) </u>
- simply run the arrow here to set everything up, no need to open this section or mess with settings (time to set up: 3-4 mins)
- make sure you have the Stable Diffusion model saved to your drive in AI/models/sd-v1-4.ckpt

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-484503e4-6d64-c69f-01b3-49822745b161)


In [ ]:
#@title Mount Google Drive and Prepare Folders
from google.colab import drive
drive.mount('/content/gdrive')
outputs_path = "/content/gdrive/MyDrive/AI/StableDiffusionBatch"
!mkdir -p $outputs_path
!mkdir -p "init_image"
print(f"Outputs will be saved to {outputs_path}")

Mounted at /content/gdrive
Outputs will be saved to /content/gdrive/MyDrive/AI/StableDiffusionBatch


In [ ]:
#@title Installation
!pip install pytorch-lightning torch-fidelity
!pip install numpy omegaconf einops kornia pytorch-lightning
!pip install albumentations transformers
!pip install ftfy jsonmerge resize-right torchdiffeq tqdm

!git clone https://github.com/CompVis/stable-diffusion
%cd stable-diffusion/
!git clone https://github.com/CompVis/taming-transformers
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/crowsonkb/k-diffusion.git

import sys
sys.path.append(".")
sys.path.append("./CLIP")
sys.path.append('./taming-transformers')
sys.path.append('./k-diffusion')

!echo '' > ./k-diffusion/k_diffusion/__init__.py

# Installation Setup for Facial Recognition when using Crop Upres Feature:
!pip install opencv-python
import cv2
!pip install numpy
!pip install matplotlib

In [ ]:
#@markdown You need to get the model weights yourself and put on Google Drive or this Colab instance
checkpoint_model_file = "/content/gdrive/MyDrive/AI/models/sd-v1-4.ckpt" #@param {type:"string"}

In [ ]:
#@title Define helper functions

import argparse, gc, json, os, random, sys, time, glob, requests
import torch
import torch.nn as nn
import numpy as np
import PIL
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from IPython.display import display, clear_output
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from torch.cuda.amp import autocast
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
#from init_image.dmdim import DDIMSampler # customer sampler
from ldm.models.diffusion.plms import PLMSSampler

from k_diffusion.sampling import sample_lms
from k_diffusion.external import CompVisDenoiser


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale

class config():
    def __init__(self):
        self.ckpt = checkpoint_model_file
        self.config = 'configs/stable-diffusion/v1-inference.yaml'
        self.ddim_eta = 0.0
        self.ddim_steps = 100
        self.fixed_code = True
        self.init_img = None
        self.n_iter = 1
        self.n_samples = 1
        self.outdir = ""
        self.precision = 'full' # 'autocast'
        self.prompt = ""
        self.sampler = 'klms'
        self.scale = 7.5
        self.seed = 42
        self.strength = 0.75 # strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.H = 512
        self.W = 512
        self.C = 4
        self.f = 8
      
def load_img(path, w, h):
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
    else:
        if os.path.isdir(path):
            files = [file for file in os.listdir(path) if file.endswith('.png') or file .endswith('.jpg')]
            path = os.path.join(path, random.choice(files))
            print(f"Chose random init image {path}")
        image = Image.open(path).convert('RGB')
    image = image.resize((w, h), Image.LANCZOS)
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

opt = config()
config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
batch_idx = 0
sample_idx = 0

def generate(opt):
    global sample_idx
    seed_everything(opt.seed)
    os.makedirs(opt.outdir, exist_ok=True)

    if opt.sampler == 'plms':
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    model_wrap = CompVisDenoiser(model)       
    batch_size = opt.n_samples
    prompt = opt.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]
    init_latent = None

    if opt.init_img != None and opt.init_img != '':
        init_image = load_img(opt.init_img, opt.W, opt.H).to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

    sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

    t_enc = int(opt.strength * opt.ddim_steps)

    start_code = None
    if opt.fixed_code and init_latent == None:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

    precision_scope = autocast if opt.precision == "autocast" else nullcontext

    images = []
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for n in range(opt.n_iter):
                    for prompts in data:
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)

                        if init_latent != None:
                            z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                            samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                    unconditional_conditioning=uc,)
                        else:

                            if opt.sampler == 'klms':
                                print("Using KLMS sampling")
                                shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                sigmas = model_wrap.get_sigmas(opt.ddim_steps)
                                model_wrap_cfg = CFGDenoiser(model_wrap)
                                x = torch.randn([opt.n_samples, *shape], device=device) * sigmas[0]
                                extra_args = {'cond': c, 'uncond': uc, 'cond_scale': opt.scale}
                                samples = sample_lms(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=False)
                            else:
                                shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                samples, _ = sampler.sample(S=opt.ddim_steps,
                                                                conditioning=c,
                                                                batch_size=opt.n_samples,
                                                                shape=shape,
                                                                verbose=False,
                                                                unconditional_guidance_scale=opt.scale,
                                                                unconditional_conditioning=uc,
                                                                eta=opt.ddim_eta,
                                                                x_T=start_code)

                        x_samples = model.decode_first_stage(samples)
                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
                        for x_sample in x_samples:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            images.append(Image.fromarray(x_sample.astype(np.uint8)))
                            try:
                              if opt.saveNumberPosition == False: # don't save the number at beginning of the file
                                filepath = os.path.join(opt.outdir, f"{batch_name}({batch_idx:03})_{sample_idx:04} {opt.seed}.png")#####
                              else: # put the number at the beginning of the filename
                                filepath = os.path.join(opt.outdir, f"{sample_idx:04} {batch_name}({batch_idx:03}) {opt.seed}.png")#####
                            except:
                                filepath = os.path.join(opt.outdir, f"{sample_idx:04} {batch_name}({batch_idx:03}) {opt.seed}.png")#####                              
                            print(f"Saving to {filepath}")
                            Image.fromarray(x_sample.astype(np.uint8)).save(filepath)
                            sample_idx += 1
    return images


Loading model from /content/gdrive/MyDrive/AI/models/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.20.mlp.fc2.weight', 'vision_model.encoder.layers.11.self_attn.v_proj.bias', 'vision_model.encoder.layers.22.self_attn.q_proj.bias', 'vision_model.pre_layrnorm.weight', 'vision_model.encoder.layers.21.layer_norm2.weight', 'vision_model.encoder.layers.12.layer_norm1.bias', 'vision_model.encoder.layers.15.self_attn.v_proj.bias', 'vision_model.encoder.layers.5.self_attn.k_proj.weight', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.23.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.mlp.fc2.weight', 'vision_model.encoder.layers.22.layer_norm1.weight', 'vision_model.encoder.layers.22.mlp.fc2.bias', 'vision_model.encoder.layers.7.self_attn.v_proj.bias', 'vision_model.encoder.layers.14.self_attn.k_proj.bias', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.12.self_attn.k_pr

In [ ]:
#@title Run this code to install Real-ESRGAN
# this is a requirement for the bonus tools - face recognition

#Before start, make sure that you choose:
#* Runtime Type = Python 3
#* Hardware Accelerator = GPU
#in the **Runtime** menu -> **Change runtime type**
#Then, we clone the repository, set up the envrironment, and download the pre-trained model.

# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# <u> Information on How to Use</u> *(DON'T RUN; this is Purely Informative / Documentation)*
- this section only contains reminders and tips on usage. Section 2. Batch Prompting is the working code

In [ ]:
'''
#@title Prompting:
#@markdown `NOTE: The settings in this informative cell are the code defaults for the most basic use.` 
<br>
#@markdown `If you have issues or want to revert, set the fields in the code cell (inside Section 2. Batch Prompting) to these values`
<br>
#@markdown `It's also best to make sure all the optional fields are set to 0, checked off, or none unless you intend to use the feature`


##1---------------------------------------------------------------------------------------------
#@markdown ---
#@markdown > <u>**Section 1): Saving/Folder Settings** *(required):* <br></u>
##@markdown Shared Prompt Ending:
#@markdown 1b) **`outputFolder`**
#@markdown - folder name where this session will store the images<br>
#@markdown - This folder will exist (by default) under content/gdrive/MyDrive/AI/StableDiffusionBatch/`
<br>
#@markdown 1b) **`saveSeperatePromptsToSubfolders`**
#@markdown - True = save each prompt in a seperate folder, first 3 strings in prompt = folder name
#@markdown - False = save all prompts results in the same outputFolder (defined above in 1a)<br>
<br>
1a) outputFolder = "upres_portraits4" #@param {type:"string"}
1b) saveSeperatePromptsToSubfolders = False#@param{type:"boolean"}
#@markdown ---


##2---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 2): Prompt Template** *(optional):*</u>
<br>
#@markdown `Note: set to 'none/custom' if you rather type your prompt details than use a preset. This is for convenience`
<br>
#@markdown `Note: adding a new preset requires these 3 steps: `
#@markdown - 1) adding a new line to the promptTemplateDict dictionary (number it correctly too)
#@markdown - 2) adding this prompt as a new drop down option in promptTemplate (in the cell code, line 31~)
#@markdown - 3) adding the dropdown text as an additional elif statement in the code (look at the pattern and repeat)
<br>
#@markdown 2a) **`promptTemplate`**
#@markdown - the core prompt templates. You can select a template to reuse the same starting word + ending strings used in the prompt
#@markdown - use these to save the inconvenience of copy/pasting the same text over and over again. focus on writing the subject instead
#@markdown - *Python structure -* **key: ['templateStartWord', 'prompt details']** | example - 10: ['tattoo design of', 'inking, drawing, detailed, art by artist']

# in addition to writing the prompts here, you need to add them to the promptTemplate #@param in the main code, otherwise it won't show up to select

promptTemplateDict = {
0: ['', 'none'],
<br>
#@markdown 2b) **`emotiveTemplate`**
#@markdown - similar to promptTemplate, these strings only go at the end of the prompt to give a consistent influence to results
#@markdown - example of use: if you want variants of all your character portraits to be zombies, or to be happy, use this feature
#@markdown - *disclaimer:* may not work as intended all the time (especially with long prompts; this text gets truncated
<br>
#@markdown 2c) **`randomEnd`**
#@markdown - text that goes at the end of all prompts. use comma to split the text and randomize between comma
<br>

2a) promptTemplate = 'none/custom' #@param ["none/custom","character - female adult", "character - male adult", "environment - scifi", "environment - fantasy", "vehicle - sketch", "vehicle - render", "style - anime keyframe (use high guidance scale of 15+)", "style - comic art inks", "style - blackwork / heavily inked tattoos", "fun - action figure character", "fun - stickers", "fun - marble statue"]
2b) emotiveTemplate = 'none' #@param ["none", "happy", "angry", "sad", "zombie", "psychedelic", "sketchy drawing style"]
2c) randomEnd = "" #@param {type:"string"}
#@markdown ---



##3---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 3): Prompt Details** *(optional - toggle on/off):* <br></u>
<br>
#@markdown `Note: Your prompt must be 77 tokens or less. Any longer and the extra text gets truncated` <br>
#<br>
#@markdown There's many options for splitting up the prompts:<br>
#<br>
#@markdown - 3a): **`useThese3PromptFields`:** includes art styles or artist names *(ex. art by  alphonse mucha)*<br>
#<br>
#@markdown  - 3b): **`randomStart`:** an initial word that is randomly selected to go at the start of the prompt<br>
#@markdown    - I recommend 2 ways to use randomStart:<br>
#@markdown  - 1) use a list of multiple emotions or elements *(ex. icy, firey, rainy)*<br>
#@markdown  - 2) leave only a single option if you have a consistent theme *(ex. 'female' or 'male')*<br>
#<br>
#@markdown - 3c):**`midDetails`:** includes keywords to create higher detailed results *(ex. highly detailed, rendered in octane)*<br>
#<br>
#@markdown - 3d): **`endingStyle`:** includes art styles or artist names *(ex. art by  alphonse mucha)*<br>
#<br>
#@markdown `NOTE: you don't have to split your prompts with these seperate fields. You can simply write the whole prompt in midDetails or in the batch prompt list too`<br>
#<br>
#<br>
#@markdown  Example of How Prompts Combine: 
#@markdown - [randomStart] + **[batchPrompt]** + [midDetails] + [endingStyle] + [randomEnd]<br>
#@markdown  - `Example 1:` [pearlescent] **[jellyfish in space],** [digital art, concept art...], [by Alex Ross...]<br>
#@markdown  - `Example 2:` [pearlescent] **[Lamborghini],** [digital art, concept art...], [by Alex Ross...]<br>
#@markdown  - `Example 3:` [glowing] **[frogs on lily pads],** [digital art, concept art...], [by Alex Ross...]<br>
#@markdown  - `Example 4:` [sparkling] **[shells on a beach],** [digital art, concept art...], [by Alex Ross...]<br>
#@markdown - Notice in the examples the first word is random and the midDetail + endingStyle words are consistent
#@markdown - The batch prompts are set up in a list a seperate cell under the variable **promptsA**<br>
#<br>

3a) useThese3PromptFields = True#@param{type:"boolean"}
3b) randomStart = "female" #@param {type:"string"}
3c) midDetails = "digital art, concept art, smooth, sharp focus, high definition, rendered in Octane" #@param {type:"string"}
3d) endingStyle = "illustration, art by artgerm and greg rutkowski and alphonse mucha" #@param {type:"string"}
#@markdown ---


##4---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 4): Basic Settings** *(required - all modes):* </u><br>
<br>
#@markdown 4a) **`width and height`:**
#@markdown - image dimensions. range of 512-768 is best for portrait. Max is 704 x 704 depending on the GPU <br>
#<br>
#@markdown 4b) **`aspectRatio`:**
#@markdown - Presets for standard image ratios to quickly choose between. Print as well as Film Ratio standards<br>
#<br>
#@markdown 4c) **`guidance_scale`:**
#@markdown - strength of text prompt. 15 is good, 20 can have greater realism. 7 was beta default. Experiment<br>

#<br>
#@markdown 4d) **`steps`:**
#@markdown - number of diffusion steps. more takes longer, but has better detail. test with 5-20 and use 50-80. rarely exceed 100 (costly and insignificant)<br>
#<br>
#@markdown 4e) **`number_of_repeats`:**
#@markdown - how many rounds of iterating through the prompt list. the prompts always run from first to last of each before repeating<br>
#<br>
#@markdown 4f) **`sampler`:**
#@markdown - it seems klms is default, but sometime of my results worked better with ddim, especially portraits<br>
#<br>
#@markdown 4g) **`seed`:**
#@markdown - set -1 to generate by random, otherwise input # seed to regenerate. Make use of randomStart/End with seed for fun variations, or use it for the sequence (same seed = consistent)<br>
#<br>
#@markdown 4h) **`first_prompt_only`:**
#@markdown - will only run on the first prompt in the prompts list<br>
#@markdown - good if you want to test on a single image without requiring the deletion of the list<br>
#<br>
4a) width = 640 #@param {type:"integer"}
4a) height = 640 #@param {type:"integer"}
4b) aspectRatio = 'custom' #@param ["custom","1:1  (square)","4:3  (standard)","16:9 (widescreen)","21:9 (ultrawide)","4:5  (instagram)","8x11 (paper standard)","2:3  (poster standard)"]
width_height = [width, height] # param{type: 'raw'}
4c) guidance_scale = 15 #@param {type:"slider", min:0, max:40, step:0.5}
4d) steps = 50 #@param {type:"integer"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
number_of_images = 1 #param {type:"integer"}
4e) number_of_repeats = 3 #@param {type:"integer"}
4f) sampler = 'ddim' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 # param {type:"number"}
4g) seed = -1 # @param {type:"integer"}
4h) firstPromptOnly = False#@param{type:"boolean"}
#@markdown ---


##5---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 5): Init Image (img2img)** *(optional):*</u><br> 
<br>
#@markdown `Note: you must upload the image(s) to the init_image folder on the left Files menu under the / content/init_image/ folder, or change the defaultInitPath`<br>
#<br>
#@markdown 5a) **`randomizeImageFromFolder:`**
#@markdown  - leave unchecked (False) to go through the list of images that are lined up with the prompt
#@markdown  - set to True to pull a random image from the **defaultInitPath** to go with your prompts`<br>
#<br>
#@markdown - some notes on randomization:<br>
#@markdown  - if you leave randomization off, make sure your count of images matches the number of prompts in both lists 10 prompts = 10 images<br>
#@markdown  - if you randomize the init_images, then you don't have to have the matching # of prompts to image. You could have 3 images in the folder and 100 prompts<br>
#<br>
#@markdown 5b) **`defaultInitPath:`**
#@markdown - where the init_images need to be saved to load. /content/ is fine most of the time
#@markdown - a custom drive path is great if you want to save collections on your drive to use randomly across different promps
<br>
#@markdown 5c) **`initial_image`:**
#@markdown - if you want to work with a single image without setting up the list array, then you can enter the image name/extension here<br>
#@markdown - otherwise you can leave this field blank<br>
#<br>
#@markdown 5d) **`init_strength`:** *IMPORTANT* I use this option the most *(since I use img2img the most)*<br>
#@markdown - Adjust the intensity of how much the input image influences the result<br>
#@markdown - 0 means no input image is used. range 0.1 - 0.3 follows loose for creativity. 0.45 - 0.7 follows image closer for consistency
#@markdown - The strength goes in increments of 0.1, however you will find that 0.05 is also a valid and different result in between 0 and 0.1!
<br>
5a) randomizeImageFromFolder = False#@param{type:"boolean"}
5b) defaultInitPath = "/content/init_image/" #@param {type:"string"}
5c) initial_image = "" #@param {type:"string"}
init_image_or_folder = "/content/init_image/" + initial_image
5d) init_strength = 0 #@param {type:"number"}
#@markdown ---



##6---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 6): Sequence Settings - Cycles** *(Through Ranges - optional):* <br></u>
<br>
#@markdown -  `Note 1: check both init_strength_0_to_1 and cfg_scale_7_to_20 to True to generate a grid of possibilities. 80 generations`<br>
#@markdown -  `Note 2: Can iterate on a seed through the scales to see how the values affect results, or randomize:`

#@markdown 6a) **`cycleThroughPromptList`:**
#@markdown - activate to cycle through the prompt list templates to generate **variations** on the same prompts
#<br>
#@markdown 6b) **`templatesToCycle`:**
#@markdown - when cycleThroughPromptList=True, these are the keys of the Prompt Templates to cycle through
#@markdown - example: 1,2 means a female and a male in a digital painting style is created for each prompt (granted you have default prompt templates)
#@markdown - the key numbers can be found labeled in the promptTemplateDict variable
#<br>
#@markdown 6c) **`init_strength_0_to_1`:**
#@markdown - Goes through image strength setting 0 - 1 to see its effects. runs only on 1st prompt
#<br>
#@markdown 6d) **`cfg_scale_7_to_20`:**
#@markdown -  Goes through guidance_scale setting 7 - 20 to see its effects. runs only on 1st prompt
#<br>
#@markdown 6e) **`init_strength_random`:**
#@markdown - randomly picks an image strength value from 0.1 to 0.7<br>
#<br>
#@markdown 6f) **`cfg_scale_random`:**
#@markdown - randomly picks an guidance_scale value from 7 to 20<br>
#<br>
6a) cycleThroughPromptList = False#@param{type:"boolean"}
6b) templatesToCycle = "1,2" #@param {type:"string"}
# these options would run 14 times:
6c) init_strength_0_to_1 = False#@param{type:"boolean"}
6d) cfg_scale_7_to_20 = False#@param{type:"boolean"}
6e) init_strength_random = False#@param{type:"boolean"}
6f) cfg_scale_random = False#@param{type:"boolean"}
save_settings_file = False#param{type:"boolean"}
#@markdown ---


##7---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 7): Sequence Settings with Images** *(optional interpolation):* <br></u>
<br>
#@markdown 7a) **`generatedImageCycle`:**
#@markdown - This will cycle through the initial prompt (if firstPromptOnly is True) or the list of prompts with slight variance, creating morphing images!
#@markdown - This can create trippy results OR be used to **create subtle variations for concept art / design!**

#<br>
#@markdown 7b) **`specifyStrengthLock`:**
#@markdown - -1 is default to go from 0.6 to 0.2; instead you can lock to only gen say all 0.05 (note: It doesn't blend as well locking to a single value. -1 is best)
#<br>
#@markdown 7c) **`usePromptsOverTimeCycle`:**
#@markdown - Set this to true to use the variable **promptInterpolateMulti** from cell 2
#@markdown - In the dictionary of promptInterpolateMulti, you'll be able to set a duration of frames for each prompt and cycle through multiple prompts!
#@markdown - This option is fantastic for animated morphs over time. It can create trippy results and unpredictable transformations
#<br>
#@markdown 7d) **`promptOverTimeSetting03InsteadOf02`:**
#@markdown - Set this to True to generate interpolations down to 0.3 instead of 0.2. 
#@markdown - This will create a more subtle interpolation which will blender better, but will not create as much interest
#<br>
# for interpolating using generated image cycles on INIT_STRENGTH:
7a) generatedImageCycle = False#@param{type:"boolean"} 
7b) specifyStrengthLock = 0.2 #@param {type:"number"} # -1 is default to go from 0.6 to 0.2; or you can lock to only gen say 0.05. It doesn't blend as well locking to a single value
7c) usePromptsOverTimeCycle = False#@param{type:"boolean"}
7d) promptOverTimeSetting03InsteadOf02 = False#@param{type:"boolean"}
#@markdown ---


##8---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 8): Research Outputs** *(optional):* <br></u>
<br>





#@markdown ---
'''

'\n#@title Prompting:\n#@markdown `NOTE: The settings in this informative cell are the code defaults for the most basic use.` \n<br>\n#@markdown `If you have issues or want to revert, set the fields in the code cell (inside Section 2. Batch Prompting) to these values`\n<br>\n#@markdown `It\'s also best to make sure all the optional fields are set to 0, checked off, or none unless you intend to use the feature`\n\n\n##1---------------------------------------------------------------------------------------------\n#@markdown ---\n#@markdown > <u>**Section 1): Saving/Folder Settings** *(required):* <br></u>\n##@markdown Shared Prompt Ending:\n#@markdown 1b) **`outputFolder`**\n#@markdown - folder name where this session will store the images<br>\n#@markdown - This folder will exist (by default) under content/gdrive/MyDrive/AI/StableDiffusionBatch/`\n<br>\n#@markdown 1b) **`saveSeperatePromptsToSubfolders`**\n#@markdown - True = save each prompt in a seperate folder, first 3 strings in prom

# <u>Prompt Storage</u> *(reference only, don't run)*
- this section is for saving useful prompts that aren't connected to code in any way, they are just here for convenience 

In [ ]:
# Put each of your prompts on a new line
# Examples:
'''
Animals:
a frog relaxing on a lilypad while ribbiting
a shiba inu riding a ufo to the moon
Characters:
wizard with a large brim magic hat and a long robe casting a spell
barbarian hunter with a club and an athletic physique
'''

# Prompt Template Brainstorming:
'''
extremely detailed oil painting, by rhads, sargent and leyendecker, savrasov levitan polenov, bruce pennington, studio ghibli, tim hildebrandt, digital art, landscape painting, trending on artstation, masterpiece
a wholesome motorcycle ride across beautiful scenery, studio Ghibli, Pixar and Disney animation, sharp, Rendered in Redshift and Unreal Engine 5 by Greg Rutkowski, Bloom, dramatic lighting
'''




# my custom prompt ideas:
'''
Cheesy Characters:
gothic, goth fantasy blacksmith apprentice with magical powers, character design, black hair, pentagram
gothic, goth fantasy blacksmith with magical powers, pentagram
barbarian hunter with a bow and arrow and an athletic physique
wizard with a large brim magic hat and a long robe casting a spell
attractive queen of the moon Athena and a starry night
female gothic crusader with her face visible, long hair, metal armor, character design, full body

Full Prompts:
female gothic crusader with her face visible, long hair, metal armor, character design, full body, intricate fine tipped pen drawing, inktober, Fine Line Tattoo, manga line art, monochrome, dotwork, by dan hilliard, by Stanislaw Wilczynski, by alphonse mucha, by aaron horkey

Animals:
a frog relaxing on a lilypad while ribbiting

Pokemon:
pokemon trainer portrait of a male forest ranger with a red hat and red jacket
pokemon trainer portrait of a male jock with spiky black hair holding up a pokeball
pokemon trainer portrait of a male electricity magician with bright yellow spiky hair juggling pokeballs
pokemon trainer portrait of a creepy scientist with big glasses and a dumb expression, blue hair
pokemon trainer portrait of a young boy who tries to look confident
pokemon trainer portrait of a badass and cool pokemon trainer with street fashion, rapper cap, holding a pokeball

Interesting Artists:
Alex Grey?? (psychedelic, trippy)
'''

'\nCheesy Characters:\ngothic, goth fantasy blacksmith apprentice with magical powers, character design, black hair, pentagram\ngothic, goth fantasy blacksmith with magical powers, pentagram\nbarbarian hunter with a bow and arrow and an athletic physique\nwizard with a large brim magic hat and a long robe casting a spell\nattractive queen of the moon Athena and a starry night\nfemale gothic crusader with her face visible, long hair, metal armor, character design, full body\n\nFull Prompts:\nfemale gothic crusader with her face visible, long hair, metal armor, character design, full body, intricate fine tipped pen drawing, inktober, Fine Line Tattoo, manga line art, monochrome, dotwork, by dan hilliard, by Stanislaw Wilczynski, by alphonse mucha, by aaron horkey\n\nAnimals:\na frog relaxing on a lilypad while ribbiting\n\nPokemon:\npokemon trainer portrait of a male forest ranger with a red hat and red jacket\npokemon trainer portrait of a male jock with spiky black hair holding up a po

# 2. <u> Batch Prompting</u> *(where the magic takes place, **EXPAND THIS IT!**)*

- Remember to run all 4 cells at the startup and the first 3 whenever you edit them!

> **Use Cases:** | txt2img (single or batch) | img2img (single or batch) | cycle through cfg/init | sequence animator | prompt templates
Note: the next code cell after this line of text must be ran. it's where the descriptive prompt lists are stored

In [ ]:
##@title # CELL 2b: MAIN PROMPTS & INITIAL IMAGES (run)
#Info = 'click Show Code to view, click Run Cell to update:' #@param ["click Show Code to view, click Run Cell to update:"]
batch_idx = 0 # some users were getting errors for this and the number can get bloated after many generations, so this is a temp solution to avoid issues with it

# IMPORTANT 1: Make sure to update this cell every time you change prompts and init_images!
# IMPORTANT 2: For the prompts and images to match, make sure to sync their position in the list

# Put each of your prompts on a new line:
promptsA = '''
a wholesome and cute cabin in the woods by Studio Ghibli
athena the goddess of the night
'''

In [ ]:
#@title # Cell 3: Main Code (the magic button; **RUN IT!**)
##@markdown

import re
import random
import os

# create a list of all the prompts from new line:
if type(promptsA) is str:
  promptsA = [i for i in promptsA.split('\n') if i]

def clean_string(s):
    s = ''.join([c for c in s if (re.match('[a-zA-Z0-9 _]', c) or ord(c) > 127)]).strip()
    if len(s) > 200:
        return (s[:150]).strip()
    return s

##1---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 1): Saving/Folder Settings** *(required):* <br></u>
outputFolder = "folderNameInsert" #@param {type:"string"}
saveSeperatePromptsToSubfolders = False#@param{type:"boolean"}
numberBeginningOfFile = True#@param{type:"boolean"}
opt.saveNumberPosition = numberBeginningOfFile
# note: the actual saving to output directory was moved lower since we might want to save the prompt name there
#@markdown ---



##2---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 2): Prompt Template** *(optional):*</u>
# <br>
#@markdown `Note: set to none if you rather type your prompt details in section 3) |  the selected template is used on EVERY prompt in the promptsA list`
promptTemplate = 'environment - fantasy' #@param ["none/custom","character - female adult", "character - male adult", "environment - scifi", "environment - fantasy", "vehicle - sketch", "vehicle - render", "design - weapon", "style - anime keyframe (use high guidance scale of 15+)", "style - comic art inks", "style - blackwork / heavily inked tattoos", "fun - action figure character", "fun - stickers", "fun - marble statue"]
templateSelected = "none"
templateSelectedStart = ""
templateSelectedKey = 0

if promptTemplate == "none/custom":
  templateSelectedKey = 0
elif promptTemplate == "character - female adult":
  templateSelectedKey = 1
elif promptTemplate == "character - male adult":
  templateSelectedKey = 2
elif promptTemplate == "environment - scifi": 
  templateSelectedKey = 3
elif promptTemplate == "environment - fantasy": 
  templateSelectedKey = 4
elif promptTemplate == "vehicle - sketch": 
  templateSelectedKey = 5
elif promptTemplate == "vehicle - render": 
  templateSelectedKey = 6
elif promptTemplate == "design - weapon": 
  templateSelectedKey = 7
elif promptTemplate == "style - anime keyframe (use high guidance scale of 15+)": 
  templateSelectedKey = 8
elif promptTemplate == "style - comic art inks": 
  templateSelectedKey = 9
elif promptTemplate == "style - blackwork / heavily inked tattoos": 
  templateSelectedKey = 10
elif promptTemplate == "fun - action figure character": 
  templateSelectedKey = 11
elif promptTemplate == "fun - stickers": 
  templateSelectedKey = 12
elif promptTemplate == "fun - marble statue": 
  templateSelectedKey = 13

# code for determining which key is selected:
# templateSelectedKey = 6
promptTemplatedSelectedlist = promptTemplateDict[int(templateSelectedKey)] # gets both the values of the selected key as a list, ex ['male', 'line art']
templateSelectedStart = promptTemplatedSelectedlist[0]
templateSelected = promptTemplatedSelectedlist[1]
# end template prompt list ------------------------------------------------------------

emotiveTemplate = 'none' #@param ["none", "happy", "angry", "sad", "zombie", "psychedelic", "sketchy drawing style"]
emoteTemplateSelected = "none"
if emotiveTemplate == "none":
  emoteTemplateSelected = "none"

elif emotiveTemplate == "happy":
  emoteTemplateSelected = "happy friendly cheerful smiling joy sparkling magical"

elif emotiveTemplate == "angry":
  emoteTemplateSelected = "villain angry pissed off frowning yelling spitting fighting violent extreme anger rage fury outrage furious open mouth"

elif emotiveTemplate == "sad":
  emoteTemplateSelected = "depressed sad saddened heartbroken pouting crying tears frown grief grieving hopeless whimpering"

elif emotiveTemplate == "zombie":
  emoteTemplateSelected = "black dark very dark black and creepy horror terrifying bloody gore wounds blood splats zombie terrified dead"

elif emotiveTemplate == "psychedelic":
  emoteTemplateSelected = "psychedelic trippy colorful neon kaleidoscope mushrooms trance bright mind-blowing mystical spiritual mysticism psilocybin hallucinogenic"

elif emotiveTemplate == "sketchy drawing style":
  emoteTemplateSelected = "line art line drawing pen and ink black and white simple simplified sketch drawing pencil drawing rough thin lines line linear etched flat drawing"

# end template emotive addition list -----------------------------------------------------------

randomEnd = "" #@param {type:"string"}
randomStringEnd = randomEnd.split(",")
numOfRandomStringEnd = len(randomStringEnd)
wordRNDMend = ""
#@markdown ---



##3---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 3): Prompt Details** *(optional - toggle on/off):* <br></u>
# <br>
#@markdown `Note: these values are included on EVERY prompt in the list if no template is used and useThese3PromptFields = True`
useThese3PromptFields = False#@param{type:"boolean"}
randomStart = "" #@param {type:"string"}
if useThese3PromptFields == False:
  randomStart = ""
wordRNDM = ""
randomStringLS = randomStart.split(",")
numOfRandomStringStart = len(randomStringLS)
midDetails = "" #@param {type:"string"}
endingStyle = "" #@param {type:"string"}
if useThese3PromptFields == False:
  midDetails = ""
  endingStyle = ""
# next we determine if we should actually use these custom strings or use our prompt templates:
promptsB = ""
if templateSelectedKey == 0:
  promptsB = [x + ", " + midDetails for x in promptsA] # add the universal style text to each prompt in the list
  prompts = [x + ", " + endingStyle for x in promptsB] # add the universal style text to each prompt in the list
else:
  prompts = [x + ", " + templateSelected for x in promptsA] # add the universal style text to each prompt in the list
#@markdown ---



##4---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 4): Basic Settings** *(required - all modes):* </u><br>
#<br>
#@markdown `Note: Defaults: width/height = 640/640 | scale = 15 | steps = 50 | repeats = 6 | sampler = ddim | seed = -1 (random)`
width = 628 #@param {type:"integer"}
height = 628 #@param {type:"integer"}
aspectRatio = '1:1  (square)' #@param ["custom","1:1  (square)","4:3  (standard)","16:9 (widescreen)","21:9 (ultrawide)","4:5  (instagram)","8x11 (paper standard)","2:3  (poster standard)"]
if aspectRatio == "custom":
  width = width
  height = height
elif  aspectRatio == "1:1  (square)":
  width = 640
  height = 640
elif  aspectRatio == "4:3  (standard)":
  width = 768
  height = 576
elif  aspectRatio == "16:9 (widescreen)":
  width = 832
  height = 480
elif  aspectRatio == "21:9 (ultrawide)":
  width = 1024
  height = 448
elif  aspectRatio == "4:5  (instagram)":
  width = 512
  height = 640
elif  aspectRatio == "8x11 (paper standard)":
  width = 512
  height = 672
elif  aspectRatio == "2:3  (poster standard)":
  width = 512
  height = 768

width_height = [width, height] # param{type: 'raw'}
guidance_scale = 16 #@param {type:"slider", min:0, max:40, step:1}
steps = 40 #@param {type:"integer"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
number_of_images = 1 #param {type:"integer"}
number_of_repeats = 10 #@param {type:"integer"}
sampler = 'ddim' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 # param {type:"number"}
seed = -1 # @param {type:"integer"}
firstPromptOnly = True#@param{type:"boolean"}
#@markdown ---




##5---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 5): Init Image (img2img)** *(optional):*</u><br> 
#<br>
#@markdown `Note: the default path is: / content/init_image/`
randomizeImageFromFolder = False#@param{type:"boolean"}
defaultInitPath = "/content/init_image/" #@param {type:"string"}
if defaultInitPath == "":
  defaultInitPath = "/content/init_image/"

# find all the images in the folder meant to read them:
numberOfImages = 0
init_imgs_list = []
for path in os.listdir(defaultInitPath):
    if os.path.isfile(os.path.join(defaultInitPath, path)):
        numberOfImages += 1
        init_imgs_list.append(path)
init_imgs_list.sort()
# print(numberOfImages)
# print(init_imgs_list)

# create a list of all the init images from new line:
if type(init_imgs_list) is str:
  init_imgs_list = [i for i in init_imgs_list.split('\n') if i]
if len(init_imgs_list) == 0:
  init_imgs_list.append("none")
  
initial_image = "" #@param {type:"string"}
checkForInitial_Image = len(initial_image)
if checkForInitial_Image == 0:
  init_image_or_folder = defaultInitPath
else:
  init_image_or_folder = defaultInitPath + initial_image
init_strength = 0 #@param {type:"number"}
#@markdown ---



##6---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 6): Sequence Settings - Cycles** *(Through Ranges - optional):* <br></u>
#<br>
#@markdown `Note: use cycleThroughPromptList to create variations with the prompt templates`
# for template cycling, ex generating both male and female or sketch after the other:
cycleThroughPromptList = False#@param{type:"boolean"}
templatesToCycle = "1,2" #@param {type:"string"}
listOfTemplatesToCycle = templatesToCycle.split(",")
numOfTemplatesToCycle = len(listOfTemplatesToCycle)
# end template cycle variables-----
# these options would run 14 times:
init_strength_0_to_1 = False#@param{type:"boolean"}
cfg_scale_7_to_20 = False#@param{type:"boolean"}
init_strength_random = False#@param{type:"boolean"}
cfg_scale_random = False#@param{type:"boolean"}
#@markdown ---


##7---------------------------------------------------------------------------------------------
#@markdown > <u>**Section 7): Sequence Settings with Images** *(optional interpolation):* <br></u>
#<br>
# forgot what the next one does, it wasn't as good as doing the image cycle though:
useGeneratedImageAsInit = False#param{type:"boolean"}
# for interpolating using generated image cycles on INIT_STRENGTH:
generatedImageCycle = False#@param{type:"boolean"} 
specifyStrengthLock = -1 #@param {type:"number"} # -1 is default to go from 0.6 to 0.2; or you can lock to only gen say 0.05. It doesn't blend as well locking to a single value
activateStrengthLock = False
if specifyStrengthLock == -1:
  activateStrengthLock = False
else:
  activateStrengthLock = True
# prompts over time lets us pick keyframe for duration of certain prompts
usePromptsOverTimeCycle = False#@param{type:"boolean"}
# sometimes there's too large a jump when we go down to 0.2 inti strngth so this has smoother results:
promptOverTimeSetting03InsteadOf02 = False#@param{type:"boolean"}
promptsOverTimeFramesList = []
promptsOverTimePromptList = []
totalFrames = 0
# if we're using prompts over time, we take the user created DICT of frame count / prompts and split to new list
# based on frame count, we'll duplicate that prompt into the prompt list the rest of the script will read
if usePromptsOverTimeCycle == True:
  firstPromptOnly = False
  prompts.clear()
  promptsA.clear()
  promptsB = ""
  for key, values in promptInterpolateMulti.items():
     number = int(values[0]) # frame count
     promptsOverTimeFramesList.append(number) # add to list of frame counts, ex. 6
     promptsOverTimePromptList.append(values[1]) # adds to list of prompt strings, ex. 'cat'
  for x in promptsOverTimeFramesList:
    totalFrames += int(x) # this value will be the total of the prompts / frames we gen
  number_of_repeats = totalFrames # we might as well pick an accurate generation count to match our frame count. so override this user inputted variable
  counta = 0
  zz = 0
  for y in range(totalFrames):
    promptToSend = promptsOverTimePromptList[zz]
    promptsA.append(promptToSend)
    counta += 1
    if counta == (promptsOverTimeFramesList[zz]):
      zz += 1
      counta = 0
  if templateSelectedKey == 0:
    promptsB = [x + ", " + midDetails for x in promptsA] # add the universal style text to each prompt in the list
    prompts = [x + ", " + endingStyle for x in promptsB] # add the universal style text to each prompt in the list
  else:
    prompts = [x + ", " + templateSelected for x in promptsA] # add the universal style text to each prompt in the list
# ----- end set up of the usePromptsOverTimeCycle / multi prop with frame count cycle interpolation setup -----------------

save_settings_file = True#param{type:"boolean"} # it seems to break script if this is set to False, so I hid it

# Sequence Counter for evaluating cfg and init_strength scales:
areBothScalesToGen = False
if init_strength_0_to_1 == True:
  firstPromptOnly = True
  if cfg_scale_7_to_20 == True:
    number_of_repeats = 80 # value if both options are selected
    areBothScalesToGen = True
  else:
    number_of_repeats = 10
if cfg_scale_7_to_20 == True:
  firstPromptOnly = True
  if init_strength_0_to_1 == True:
    number_of_repeats = 80 # value if both options are selected
    areBothScalesToGen = True
  else:
    number_of_repeats = 8
counterCFG = 6
counterINITSTRENGTH = 0
counter = 0
countRepeats = -1
number_of_repeats_display = number_of_repeats
randomNumber = 0
countPromptTemplateCycles = -1
countSubfolderNumber = 0
sample_idx = 1 # reset this number every generation

# INIT_Image gen from drive:
activateUsePreviousGenImage = False # we have to activate this later in the code
areWeOnOddInitGenCycle = True # odd will gen random seed at 0.3, even will fen previous seed at 0.7
seedA_GenCycle = random.randint(0, 2**32) if seed == -1 else seed
seedB_GenCycle = random.randint(0, 2**32)
reset_init_strength_for_gen_cycle = False
if generatedImageCycle == True:
  init_strength = 0.9 #starting value for cycle; we go from 0.6 down to 0.2 in 0.1 increments
  init_strength_0_to_1 = False
  cfg_scale_7_to_20 = False

if firstPromptOnly == True:
  number_of_repeats_display = number_of_repeats
else:
  number_of_repeats_display = number_of_repeats * len(prompts)

# check if we're cycling through prompt templates:
if cycleThroughPromptList == True:
  if init_strength_0_to_1 == False and cfg_scale_7_to_20 == False:
    if usePromptsOverTimeCycle == False:
      number_of_repeats = numOfTemplatesToCycle * len(prompts)
      number_of_repeats_display = number_of_repeats
    elif usePromptsOverTimeCycle == True:
      number_of_repeats = numOfTemplatesToCycle
      number_of_repeats_display = number_of_repeats

if usePromptsOverTimeCycle == True:
  number_of_repeats_display = number_of_repeats
  number_of_repeats = 1 # we only repeat once since we already compiled all our prompts

# --------- CODE STARTS GENERATING ----------------
for x in range(number_of_repeats):
  countRepeats +=1
  countSubfolderNumber = 0
  # These counters count every time the prompt is sent for another repeat
  if firstPromptOnly == True:
    if init_strength_0_to_1 == False or cfg_scale_7_to_20 == False:
      counter = counter
  else:
    if init_strength_0_to_1 == False:
      counter = 0
    if cfg_scale_7_to_20 == False:
      counter = 0
  if init_strength_0_to_1 == True or cfg_scale_7_to_20 == True:
    if countRepeats == number_of_repeats:
      break

  # check if we're cycling through prompt templates and need to swap the prompt text:
  if cycleThroughPromptList == True:
    countPromptTemplateCycles += 1
    if templateSelectedKey == number_of_repeats: # prevents the list going out of range
      break
    try:
      templateSelectedKey = listOfTemplatesToCycle[countPromptTemplateCycles] # for some reason this line turns the into to string, so have to turn it back to int
    except:
      break
    promptTemplatedSelectedlist = promptTemplateDict[int(templateSelectedKey)] # gets both the values of the selected key as a list, ex ['male', 'line art']
    templateSelectedStart = promptTemplatedSelectedlist[0]
    templateSelected = promptTemplatedSelectedlist[1]
    prompts = [x + ", " + templateSelected for x in promptsA] # add the universal style text to each prompt in the list


  # calculating the full prompt: ----------------------------------
  for prompt in prompts:
    # check if we are only supposed to generate the first prompt or all them:
    if firstPromptOnly == True:
      if prompt == prompts[1]:
        break

    # the random start and end words are broken, for some reason
    if numOfRandomStringStart != 0 and numOfRandomStringStart != 1:
      # wordRNDM = random.choice(randomStringLS) # trash line didn't run properly
      number = ((seed - seed) + (numOfRandomStringStart - 1))
      randomNumber = random.randint(0, number)
      # print(numOfRandomStringStart)
      if randomNumber > numOfRandomStringStart or randomNumber <0:
        randomNumber = 0
      wordRNDM = randomStringLS[randomNumber]
    elif numOfRandomStringStart == 1:
      wordRNDM = randomStringLS[0]
    #print(randomNumber)
    #print(wordRNDM)

    if numOfRandomStringEnd != 0 and numOfRandomStringEnd != 1:
      # wordRNDMend = random.choice(randomStringEnd) # this trash didn't work for some reason
      number = ((seed - seed) + (numOfRandomStringEnd - 1))
      randomNumber = random.randint(0, number)
      if randomNumber > numOfRandomStringEnd or randomNumber <0:
        randomNumber = 0
      wordRNDMend = randomStringEnd[randomNumber]
    elif numOfRandomStringEnd == 1:
      wordRNDMend = randomStringEnd[0]

    if templateSelected != "none":
      randomStringLS = templateSelectedStart
      wordRNDM = templateSelectedStart

    # opt.prompt = prompt
    if numOfRandomStringStart == 0:
      if numOfRandomStringEnd == 0:
        promptPreFinal = prompt
      else:
        if len(wordRNDMend) == 0:
          promptPreFinal = prompt
        else:
          promptPreFinal = prompt + "," + wordRNDMend
    else:
      if numOfRandomStringEnd == 0:
        promptPreFinal = wordRNDM + ", " + prompt
      else:
        if len(wordRNDMend) == 0:
          promptPreFinal = wordRNDM + ", " + prompt
        else:
          promptPreFinal = wordRNDM + ", " + prompt + "," + wordRNDMend

    # finally bring the combined prompt together:
    if emoteTemplateSelected == "none":
      opt.prompt = promptPreFinal
    else:
      opt.prompt = promptPreFinal + " " + emoteTemplateSelected

    shortenedPromptNameList = prompt.split()[:3]
    shortenedPromptNameStr = " ".join(shortenedPromptNameList)
    batch_name = clean_string(shortenedPromptNameStr) # shorten the prompt word since the prompts are too long for windows!
    if generatedImageCycle == True:
      amountOfPaddingForN = len(str(number_of_repeats)) # how much padding to get
      xNum = f'{sample_idx:{0}{amountOfPaddingForN}}'
      paddedBatchName = str(xNum) + " " + batch_name
      batch_name = paddedBatchName

    # Related to INIT IMAGES:
    if init_imgs_list[0] != "none":
      # opt.outdir = os.path.join(outputs_path, batch_name) # if you want output dir to match the prompt
      if randomizeImageFromFolder == True and useGeneratedImageAsInit == False and generatedImageCycle == False:
        opt.init_img = init_image_or_folder
      if randomizeImageFromFolder == False and generatedImageCycle == False:
        if checkForInitial_Image == 0: # we have to check if user wants to run a single specific image (meaning the field in the init image area)
          if firstPromptOnly == False:
            if init_strength == 0:
              print("")
              # full_init_img_path = init_image_or_folder + init_imgs_list[counter]
            else:
              full_init_img_path = init_image_or_folder + init_imgs_list[counter]
          else:
            full_init_img_path = init_image_or_folder + init_imgs_list[0]
        else:
          full_init_img_path = init_image_or_folder
        print("Init image used: " + full_init_img_path)
        opt.init_img = full_init_img_path


    opt.ddim_steps = steps
    opt.n_iter = 1
    opt.n_samples = samples_per_batch

    # seed creation and the cycle init image gen from itself:
    if generatedImageCycle == False:
      # pick random seed for images or the seed specified:
      opt.seed = random.randint(0, 2**32) if seed == -1 else seed
    else:
      # this only activates if we are trying to cycle through INIT image and reuse it from the generated location:
      # first gen only:
      if activateUsePreviousGenImage == False:
        opt.init_img = init_image_or_folder # do only on the first generation to get our input image
        activateUsePreviousGenImage = True

      # core cycle:
      init_strength_endPoint = 0.2 # how far we go from 0.6 to 0.2 or 0.3
      if promptOverTimeSetting03InsteadOf02 == True:
          init_strength_endPoint = 0.3 # for a more subtle interpolation
      if areWeOnOddInitGenCycle == True:
        opt.seed = seedA_GenCycle
        if activateStrengthLock == False:
          init_strength -= 0.1 # only go down 0.1 if there is no strength lock
          round(init_strength)
          if init_strength == init_strength_endPoint or init_strength <= (init_strength_endPoint + 0.1): # when we get to the end; either 0.2 or 0.3
            areWeOnOddInitGenCycle = False
            reset_init_strength_for_gen_cycle = True
            seedA_GenCycle +=1
        elif activateStrengthLock == True:
          areWeOnOddInitGenCycle = False
          init_strength = specifyStrengthLock
          reset_init_strength_for_gen_cycle = True
          seedA_GenCycle +=1

      elif areWeOnOddInitGenCycle == False:
        # opt.seed is previous result
        opt.seed = seedB_GenCycle
        if activateStrengthLock == False:
          init_strength -= 0.1
          round(init_strength)
          if init_strength == init_strength_endPoint or (init_strength_endPoint + 0.1):
            areWeOnOddInitGenCycle = True
            reset_init_strength_for_gen_cycle = True
            seedB_GenCycle +=1
        elif activateStrengthLock == True:
            areWeOnOddInitGenCycle = True
            init_strength = specifyStrengthLock
            reset_init_strength_for_gen_cycle = True
            seedB_GenCycle +=1     
          # ---------- END THE CYCLE THROUGH INIT --------------------------
      
    # choose to pull one of the generated images as INIT Image:
    if useGeneratedImageAsInit == True and generatedImageCycle == False:
      if activateUsePreviousGenImage == False:
        activateUsePreviousGenImage = True
      elif activateUsePreviousGenImage == True:
        # this will only run when activateUsePreviousGenImage = True as set above
        if generatedImageCycle == False:
          while not os.path.exists(lastGeneratedImagePath):
            time.sleep(1) 
          if os.path.isfile(lastGeneratedImagePath):
            opt.init_img = lastGeneratedImagePath
            seed += 1 # this technique only works if you use different seeds

    # CFG Calculation:    
    opt.sampler = sampler
    if cfg_scale_7_to_20 == False:
      opt.scale = guidance_scale
    else:
      opt.scale = counterCFG
    if cfg_scale_random == True:
      opt.scale = random.uniform(7, 20)

    # INIT IMAGE:
    if init_strength_0_to_1 == True:
      init_strength = counterINITSTRENGTH
    if init_strength_random == True:
      init_strength = random.uniform(0.1, 0.8)
    opt.strength = max(0.0, min(1.0, 1.0 - init_strength))

    #opt.W = width_height[0]
    #opt.H = width_height[1]
    opt.W, opt.H = map(lambda x: x - x % 64, (width_height[0], width_height[1])) # resize to integer multiple of 64

    # Counters:
    counter += 1 # I added this to keep track of INIT images
    if firstPromptOnly == True:
      if cycleThroughPromptList == True or usePromptsOverTimeCycle == True:
        print("Generations processed: " + str(counter) + " / " + str(numOfTemplatesToCycle))
      else: 
        print("Generation processing: " + str(counter) + " / " + str(number_of_repeats_display))    
    else:
        displayCounter = (len(prompts) * countRepeats) + counter
        if cycleThroughPromptList == True:
          print("Generations processed: " + str(counter) + " / " + str(numOfTemplatesToCycle * len(prompts)))
        else:        
          print("Generation processing: " + str(displayCounter) + " / " + str(number_of_repeats_display))
    if areBothScalesToGen == False:
      counterCFG += 2 # go up from 7 to 20 if activated
      counterINITSTRENGTH = counterINITSTRENGTH + 0.1 # go up from 0.1 to 0.8 if activated
    else:
      counterINITSTRENGTH = counterINITSTRENGTH + 0.1
      if counterINITSTRENGTH > 0.9:
        counterINITSTRENGTH = 0 # reset since we're now going to go to next CFG
        counterCFG += 2 # go up from 7 to 20 if activated

    if opt.strength >= 1 or init_image_or_folder == None or opt.strength <= 0:
        opt.init_img = ""

    if opt.init_img != None and opt.init_img != '':
        opt.sampler = 'ddim'

    if opt.sampler != 'ddim':
        opt.ddim_eta = 0.0

    # debugging during processing:
    print("prompt: " + opt.prompt[:110] +"...")

    # save settings
    settings = {
        'ddim_eta': ddim_eta,
        'guidance_scale': guidance_scale,
        'init_image': init_image_or_folder,
        'init_strength': init_strength,
        'number_of_images': number_of_images,
        'prompt': opt.prompt,
        'sampler': sampler,
        'samples_per_batch': samples_per_batch,
        'seed': opt.seed,
        'steps': steps,
        'width': opt.W,
        'height': opt.H,
    }
    # saving and making output directories:
    countSubfolderNumber += 1
    if saveSeperatePromptsToSubfolders == False:
      opt.outdir = os.path.join(outputs_path, outputFolder)
    else:
      numToUsePath = countSubfolderNumber.zfill(3)
      opt.outdir = os.path.join(outputs_path, outputFolder + "/" + str(numToUsePath) + " " + shortenedPromptNameStr)

    os.makedirs(opt.outdir, exist_ok=True)
    # filePathToSaveTo = opt.outdir + "/" + batch_name + "" + batch_idx.zfill(3)  + "_" + sample_idx.zfill(4) + " " + {opt.seed} + ".png"
    pathBatchIDPad = (f'{batch_idx:03}')
    pathsample_idx = (f'{sample_idx:04}') 
    if numberBeginningOfFile == False:
      lastGeneratedImagePath = opt.outdir + "/" + batch_name + "(" + str(pathBatchIDPad)  + ")_" + str(pathsample_idx) + " " + str(opt.seed) + ".png"
    else:
      lastGeneratedImagePath = opt.outdir + "/" + str(pathsample_idx) + " " + batch_name + "(" + str(pathBatchIDPad)  + ")" + " " + str(opt.seed) + ".png"
    # while os.path.isfile(f"{opt.outdir}/{batch_name}({batch_idx})_{sample_idx:04}_settings.txt"):
        #countSubfolderNumber += 1
        # print( "counting the prompt: " + str(countSubfolderNumber))
    if save_settings_file:
      if numberBeginningOfFile == False:
        with open(f"{opt.outdir}/{batch_name}({batch_idx})_{sample_idx:04}_settings.txt", "w+", encoding="utf-8") as f:
            json.dump(settings, f, ensure_ascii=False, indent=4)
      else:
        with open(f"{opt.outdir}/{sample_idx:04} {batch_name}({batch_idx})_settings.txt", "w+", encoding="utf-8") as f:
            json.dump(settings, f, ensure_ascii=False, indent=4)
    with open(f"{opt.outdir}/prompt.txt", 'w') as f:
          f.write(opt.prompt)
    # sample_idx = 0

    for i in range(number_of_images):
        gc.collect()
        torch.cuda.empty_cache()

        images = generate(opt)

        clear_output(wait=True)
        print(f"Used seed: {opt.seed}")
        if firstPromptOnly == True:
          if cycleThroughPromptList == True or usePromptsOverTimeCycle == True:
            print("Generations processed: " + str(counter) + " / " + str(numOfTemplatesToCycle))
          else:
            print("Generations processed: " + str(counter) + " / " + str(number_of_repeats_display))
        else:
          if cycleThroughPromptList == True:
            print("Generations processed: " + str(counter) + " / " + str(numOfTemplatesToCycle * len(prompts)))
          else:
            print("Generations processed: " + str(displayCounter) + " / " + str(number_of_repeats_display))
        # print(f"Saved to: {opt.outdir}")
        print("prompt: " + opt.prompt[:110] +"...")
        # print(lastGeneratedImagePath)
        # print ("countdown of generatedImageCycle: " + str(init_strength))

        #for image in images:
        #    display(image)
        display(images[0])

        batch_idx += 1
        sample_idx += 1
        # opt.seed += 1

    # change some settings like swap to previous generated image to cycle though generatedImageCycle:  
    if generatedImageCycle == True:
      if reset_init_strength_for_gen_cycle == True:
        reset_init_strength_for_gen_cycle = False
        init_strength = 0.7
        print("wait while loading... " + lastGeneratedImagePath)
        while not os.path.exists(lastGeneratedImagePath):
          time.sleep(1) 
        if os.path.isfile(lastGeneratedImagePath):
          opt.init_img = lastGeneratedImagePath


## Additional Prompt Cells (run)
- (run once at start and whenever you edit these)

In [ ]:
#@title # CELL 1: PROMPT TEMPLATE (run)
Info = 'click Show Code to view, click Run Cell to update:' #@param ["click Show Code to view, click Run Cell to update:"]

# Here you can save prompt templates for reusing:
# if you want to edit or add your own to the template, make sure to respect Python dict: key: ['templateStartWord', 'prompt details'],
# in addition to writing the prompts here, you need to add them to the promptTemplate #@param in the main code, otherwise it won't show up to select

promptTemplateDict = {
0: ['', 'none'],
1: ['female', 'portrait, young woman, detailed gorgeous face, digital art, painting, artstation, concept art, smooth, sharp focus, high definition, detailed, illustration, art by artgerm and greg rutkowski and alphonse mucha'],
2: ['male', 'portrait, mature man, detailed rugged face, digital art, painting, artstation, concept art, smooth, sharp focus, high definition, detailed, illustration, art by J. C. Leyendecker and greg rutkowski and Norman Rockwell'],
3: ['science fiction', 'scifi, robotics, fururistic decor, trending on artstation, global illumination, matte painting, high detail, unreal engine 5, halo, star citizen, star wars, by sparth, octane render, by ralph McQuarrie, by Syd Mead, by ryan church'],
4: ['fantasy', 'painterly, nature, natural, guild wars, magical, ethereal, highly detailed, historical, period piece, by greg rutkowski, by Daniel Dociu, by noah bradley, by howard lyon'],
5: ['lineart drawing', 'drawn by bernie wrightson, industrial design sketch, sketch by Scott Robertson'],
6: ['Redshift render', 'sharp, rendered in octane, highly detailed, minimalistic, rendered in unreal engine 5, product render'],
7: ['hard surface, weapon design, scifi gun, halo, digital art, painting, artstation, concept art, smooth, sharp focus, high definition, detailed, illustration, art by greg rutkowski'],
8: ['a wholesome animation key shot of', 'studio ghibli, pixar and disney animation, sharp, rendered in unreal engine 5, anime key art by greg rutkowski, bloom, dramatic lighting'],
9: ['a scene of', 'black and white, comic book art, by yoji shinkawa and takehiko inoue and kim jung gi, masterpiece, perfect'],
10: ['intricate fine tipped pen drawing of a', 'inktober, Fine Line Tattoo, manga line art, monochrome, dotwork, by dan hilliard, by Stanislaw Wilczynski, by alphonse mucha, by aaron horkey'],
11: ['full body 3d render of', 'as a vinyl action figure, studio lighting, white background, blender, trending on artstation, 8k, highly detailed, rendered in redshift, studio product photograph'],
12: ['sticker of', 'cute sticker decal design, highly detailed, high quality, digital painting'],
13: ['marble statue', 'sculpture, marble, ancient greek statue, museum statue from stone, stone sculpt'],
}

In [ ]:
#@title #CELL 2a: INTERPOLATION PROMPTS (run this cell & the next one)
Info = 'click Show Code to view, click Run Cell to update:' #@param ["click Show Code to view, click Run Cell to update:"]

# use this section to change the prompt being interpolated over time (frames held by prompt)

# More Information:
# key#: [frameCount, promptString]
# at 30% playback speed of 30FPS video, #10 frames = 1 second
# best frame length 0.2: initial prompt = chaneg on 8 or 13 (5 increments); after change on 5,10,15, etc (this will change prompt as soon as we land back on 0.6, granted we go down to 0.2)
# best frame length 0.3: initial prompt = change on 7 or 11 (4 increments); after change on 4,8,12, etc

promptInterpolateMulti = {
0: [10, 'a frog relaxing on a lilypad while ribbiting'],
1: [10, 'a frog transforming into a cat, white background'],
2: [10, 'a cat hissing and growling in a creepy forest, white background'],
3: [10, 'a cat growing into a massive lion, white background'],
4: [10, 'a lion roaring, white background'],
5: [20, 'a exploding, explosion, burst of flames'],
6: [10, 'a face of a beautiful woman turning into a flower'],
7: [30, 'flower pedals transforming into a swarm of butterflies'],
8: [20, 'flower pedals transforming into a swarm of butterflies'],
9: [20, 'butterflies shrinking into tiny stars in a starry night'],
}

# Bonus Tools *(stand alone by section header)*
- | Upscaling (HD+ Tiles) | Facial Recognition | Upscaling (Real ESRGAN) |

## <u> A) Post Processing - **Grid Upscaling** </u> *(Tile Generation)* 

In [ ]:
# CODE BLOCK 1) Split an image into grids of 512x512 size
#@title 1) Upscaling and HD Resolution Tile Generator:
#@markdown `Note: this script requires a final image as input. It'll detail 512x512 grids across the input image, so your image should be sized up already to 2048+`<br>

from PIL import Image, ImageDraw, ImageFilter
from itertools import product
from os import listdir
from os.path import isfile, join
import os
import math
import glob
from google.colab import output
import shutil

inputImage = "landscape2.png" #@param {type:"string"}
inputDirectory = "/content/init_image" #@param {type:"string"}
outputDirectory = "/content/HD/preProcessedGridSplits"
outputDirectoryFiles = glob.glob('/content/HD/preProcessedGridSplits/**/*', recursive=True)
#saveSeperatePromptsToSubfolders = False#@param{type:"boolean"}

def splitImageTiles(filename, dir_in): # (inputImage, inputDirectory)
    # setup:
    d = 512 # the width / height we'll split our image into; it's 512 since this is the ideal size to input anyways & the math ONLY works with 512x512 size
    # dir_out = dir_in + "\\" + "outSplitGrids"
    dir_out = outputDirectory

    # clean up existing files in the output Directory:
    for f in outputDirectoryFiles:
        try:
            os.remove(f)
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))

    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size # image size, ex (2048, 3072)
    #print(w)

    # modify the width of the image to work well with the code as a power of 256:
    ratioSize = d/2 # the ratio we want to match is half of our image width
    nearest_multipleOf256 = int(ratioSize * round(w/ratioSize))
    wpercent = (nearest_multipleOf256 / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((nearest_multipleOf256, hsize), Image.ANTIALIAS) # resize the image so that the width at least matches clean to our grid
    # img.save(dir_in + 'resized_image.png')
    w, h = img.size # image size, ex (2048, 3072)
    outputGridImagesList = []

    ratioW = w / d
    ratioH = h / d
    ratioHInitial = ratioH

    ratioW = int(ratioW) # for example 4
    ratioH = int(ratioH) # for example 6
    # for some reason int rounds them all down, so we have to accomodate for these ratio possibilities
    # 1) images can be less than 0.05 difference, 2) images can be between 0.1 and 0.5 difference and 3) images can be greater than (0.5-0.99) difference
    cutsW = ratioW*2 # the amount of cut images on the wdith; the columns ; for example 8
    if ratioHInitial - ratioH < 0.05:
        cutsH = ratioH*2 # the amount of cut images on the height; the rows ; for example 12
    elif ratioHInitial - ratioH >= 0.5:
        cutsH = int((ratioH*2) + 2) # the amount of cut images on the height; the rows ; for example 12
    else:
        cutsH = int((ratioH*2) + 1)
    countW = 0
    countH = 1 # we start with 1 since we will start in the first row and we're counting by width / countW
    image_counter = 0

    # before processing the images, check if a folder to store the new images exists:
    isExist = os.path.exists(dir_out)
    if not isExist:
        os.makedirs(dir_out)    

    # grid = product(range(0, h-h%d, int(d/2)), range(0, w-w%d, int(d/2)))
    grid = product(range(0, h*2-h%d, int(d/2)), range(0, w-w%d, int(d/2)))
    for i, j in grid:
        countW += 1
        #print(countW)
        box = (j, i, j+d, i+d)
        # out = os.path.join(dir_out, f'{name}_{i}_{j}{ext}')
        # out = os.path.join(dir_out, f'{name}_{countW}_{countH}{ext}')
        out = os.path.join(dir_out, f'preGrid_{str(countH).zfill(3)}_{str(countW).zfill(3)}{ext}')
        img.crop(box).save(out)
        outputGridImagesList.append(out)

        if countW == cutsW:
            countW = 0
            countH += 1

        if countH == (cutsH + 1):
            output.clear()
            image_counter +=1
            print("Split: " + str(image_counter) + " pieces split")
            print("Finished processing! Now write descriptions for each section:")
            break
      
        output.clear()
        image_counter +=1
        print("Split: " + str(image_counter))
    
    return w, h, image_counter, cutsW, outputGridImagesList

# ------------------------------------------------------------------------
# Core Code:
# 1) Split the images into seperate grids which we'll feed into Stable Diffusion:
widthAndHeight = splitImageTiles(inputImage,inputDirectory) # run the code
initialImageWidth = (widthAndHeight[0])
initialImageHeight = (widthAndHeight[1]) # we especially need this for later
totalSplitCount = (widthAndHeight[2]) # to determine if we have the correct # of images
numberOfImageAlongX = (widthAndHeight[3]) # check how many split cells go along the X axis, to check when we reach the end later
listOfOutputImages = (widthAndHeight[4]) # the list of the output images by their path, ex: '/content/preProcessedGridSplits/preGrid_001_001.png'



# -----------------------------------------------------------------------
# 2) Get the user to input their desired text to describe each image block:
import ipywidgets as widgets
from IPython.display import display
import time

imageDescriptionsList = []
img = Image.open(os.path.join(listOfOutputImages[0]))
time.sleep(1)

if os.path.exists("/content/HD/text/") == False:
  os.makedirs("/content/HD/text/")
# note: the following will wipe out the input textfile every time we run code:
with open('/content/HD/text/descriptionsInput.txt', 'w') as f:
  f.write('')
filePathTextToUse = '/content/HD/text/descriptionsInput.txt'

for num in range((totalSplitCount - 1)):
  print ("Describing: " + str(num+1) + " / " + str (totalSplitCount))
  print ("NOTE: If you wish to use a txt file instead of typing here, type EXIT")
  display(img)
  time.sleep(0.3)
  imageDescription = input('Enter a description for this image:\n')
  if imageDescription == "EXIT":
    clear_output()
    print ("Cancelled typing prompt descriptions")
    time.sleep(0.1)
    differentTextFileToUse = input('Type the full path of the text file you wish to use:\n')
    filePathTextToUse = differentTextFileToUse
    break
  else:
    imageDescriptionsList.append(imageDescription)
    with open('/content/HD/text/descriptionsInput.txt', 'a') as f: 
      f.write(imageDescription)
      f.write('\n')
    clear_output()
    nextImage = num+1
    img = Image.open(os.path.join(listOfOutputImages[nextImage]))
    time.sleep(0.1)

    with open('/content/HD/text/descriptionsInput.txt', 'a') as f: 
      f.write(imageDescription)

shutil.copy('/content/HD/text/descriptionsInput.txt', '/content/HD/text/descriptionsInput_AUTOBACKUP.txt') # Use the shutil module
print("done")

# now we read the file we want to use as the text prompter and turn it into a list:
# fileText = open(filePathTextToUse, 'r')
with open(filePathTextToUse) as f:
    Lines = [Line.rstrip() for Line in f]
promptDescriptionsToAppend = Lines # here's the list of lines we'll use to go with the images


# -----------------------------------------------------------------------
# 3) Now we can run these prompts and images through Stable Diffusion to Upres them!


#@markdown Basic settings:
batch_name = "testsB" #@param {type:"string"}
gridRatio = "672" #@param [512, 608, 640, 672, 704]
width_height = [int(gridRatio), int(gridRatio)] #param{type: 'raw'}
guidance_scale = 25 #@param {type:"number"}
steps = 40 #@param {type:"integer"}
init_strength = 0.8 #@param {type:"number"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
num_batch_images = 1 #param {type:"integer"}
sampler = 'ddim' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 #param {type:"number"}
seed = 0 #@param {type:"integer"}

#@markdown Prompt
promptStart = "forest environment design" #@param {type:"string"}
promptEnd = "painterly, nature, natural, guild wars, magical, ethereal, highly detailed, historical, period piece, by greg rutkowski, by Daniel Dociu, by noah bradley, by howard lyon" #@param {type:"string"}
promptList = []

# append the text strings to an array to combine with full prompt later:
for x in promptDescriptionsToAppend:
  promptList.append(promptStart + ", " + x + ", " + promptEnd)

# check if we actually wrote enough lines in our text file, if not we'll add empty strings:
if len(promptList) != totalSplitCount:
  extraLinesNeeded = totalSplitCount - len(promptList)
  for x in range(extraLinesNeeded):
    promptList.append(promptStart + ", " + promptEnd)
# ------ NOW WE HAVE OUR FULL TEXT FOR THE PROMPTS! ------------------------------


# --------------------------------------------------------------------------------
# now get the input images from the folder we created earlier:
# the folder is "/content/HD/preProcessedGridSplits"
defaultInitPath = "/content/HD/preProcessedGridSplits/"

# find all the images in the folder meant to read them:
numberOfImages = 0
init_imgs_list = []
for path in os.listdir(defaultInitPath):
    if os.path.isfile(os.path.join(defaultInitPath, path)):
        numberOfImages += 1
        init_imgs_list.append(path)
init_imgs_list.sort()
#print(numberOfImages)
#print(init_imgs_list)

# NOW WE HAVE ALL THE VARIABLES WE NEED! we have the list of images, prompts and core numbers like image height and number of tiles along width!
# core variables:
# promptList (lsit of the full prompts including both custom and appended style strings)
# init_imgs_list (list of all the image paths)
# totalSplitCount (total number of splits we made = total images to process)
# initialImageWidth (in pixels of input image preprocessed)
# initialImageHeight (we especially need this for later, this is the input image height in pixels)
# numberOfImageAlongX  (check how many split cells go along the X axis)

# set up counters and the output folder:
batch_idx = 0
sample_idx = 0
# numberPosition = True
# opt.saveNumberPosition == numberPosition
outputDirectoryProcessed = "/content/HD/postProcessedGridSplits/"
outputDirectoryFiles = glob.glob('/content/HD/postProcessedGridSplits/**/*', recursive=True)
if os.path.exists(outputDirectoryProcessed) == False:
  os.makedirs(outputDirectoryProcessed)
# clean up existing files in the output Directory:
for f in outputDirectoryFiles:
    try:
        os.remove(f)
    except OSError as e:
        pass

# --------------------------------------------------------------------------------
# now get to prepare the images, prompts and settings for Stable Diffusion:
for num in range((totalSplitCount)):
  print("Processing image: " + (str(num+1)) + " / " + str(totalSplitCount))
  opt.init_img = ("/content/HD/preProcessedGridSplits/" + str(init_imgs_list[int(num)]))
  opt.ddim_steps = steps
  opt.n_iter = 1
  opt.n_samples = samples_per_batch
  # opt.outdir = os.path.join(outputs_path, batch_name)
  opt.outdir = outputDirectoryProcessed
  opt.prompt = promptList[int(num)]
  opt.sampler = sampler
  opt.scale = guidance_scale
  opt.seed = random.randint(0, 2**32) if seed == -1 else seed
  opt.strength = max(0.0, min(1.0, 1.0 - init_strength))
  opt.W, opt.H = map(lambda x: x - x % 64, (width_height[0], width_height[1])) # resize to integer multiple of 64

  if opt.init_img != None and opt.init_img != '':
      opt.sampler = 'ddim'

  if opt.sampler != 'ddim':
      opt.ddim_eta = 0.0

  # save settings
  settings = {
      'ddim_eta': ddim_eta,
      'guidance_scale': guidance_scale,
      'init_image': init_imgs_list[num],
      'init_strength': init_strength,
      'prompt': promptList[num],
      'sampler': sampler,
      'samples_per_batch': samples_per_batch,
      'seed': opt.seed,
      'steps': steps,
      'width': opt.W,
      'height': opt.H,
  }
  os.makedirs(opt.outdir, exist_ok=True)
  #with open(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt", "w+", encoding="utf-8") as f:
  #    json.dump(settings, f, ensure_ascii=False, indent=4)
  sample_idx = 0

  gc.collect()
  torch.cuda.empty_cache()

  images = generate(opt)

  clear_output(wait=True)
  print(f"Used seed: {opt.seed}")
  print(f"Saved to: {opt.outdir}")
  print("Processed image: " + str((num+1)) + " / " + str(totalSplitCount))
  display(images[0])

  batch_idx += 1
  sample_idx += 1
# ---------------- end of Stable Diffusion Code-----------------------
# now some clean up before extending and merging the images:

# clean up existing files again:
finalOutputDirectory = "/content/HD/extendedTiles/"
outputDirectoryFiles = glob.glob('/content/HD/extendedTiles/**/*', recursive=True)
if os.path.exists(finalOutputDirectory) == False:
  os.makedirs(finalOutputDirectory)
# clean up existing files in the output Directory:
for f in outputDirectoryFiles:
    try:
        os.remove(f)
    except OSError as e:
        pass

directoryToRename = "/content/HD/postProcessedGridSplits/"
directoryToReference = "/content/HD/preProcessedGridSplits/"
# rename the images to clean them up:
listOfImagesToRename = [f for f in listdir(directoryToRename) if f.endswith(".png") and isfile(join(directoryToRename, f))]
listOfImagesToRename.sort()
listOfImagesReference = [f for f in listdir(directoryToReference) if f.endswith(".png") and isfile(join(directoryToReference, f))]
listOfImagesReference.sort()
counter = 0
for x in listOfImagesToRename:
  newNameSplit = listOfImagesReference[counter].split('pre', 1)
  newName = "post" + newNameSplit[1]
  src =f"{directoryToRename}/{x}" # the incorrect file names we need to rename
  dst =f"{directoryToRename}/{newName}"
  os.rename(src, dst)
  counter += 1


# -----------------------------------------------------------------------
# 4) After upressing each tile with Stable Diffusion, we 'simply' expand them to full ratio and merge into a final image

def add_margin(directory, columnsNumber):
    # first gather variable information from the images in our folder:
    listOfImages = [f for f in listdir(directory) if f.endswith(".png") and isfile(join(directory, f))]
    listOfImages.sort()
    imgStudy = Image.open(directory + '/' + listOfImages[0]) # load the image
    w, h = imgStudy.size
    imageWidth = w # could be 512, 640, etc, it reads the actual image width so we don't guess
    numberOfImages = len(listOfImages) # total images in the folder we're reading
    extendAmount = int(imageWidth / 2) # example; if the image width is 512, then this extend amount is 256
    # extensionWidth = int(imageWidth * (columnsNumber / 2)) # the final image width, example 2048
    # extensionHeight = int((numberOfImages / (columnsNumber)) * extendAmount) # the final image height, example 3072
    
    # these values come from the actual image, so it's more legit:
    # since we have a feature called gridRatio which actually changes the final width, we'll have to accomodate new potential sizes:
    if imageWidth == 512:
      extensionWidth = initialImageWidth
      extensionHeight = initialImageHeight
    else:
      pixelDensityIncrease = imageWidth / 512
      extensionWidth = int(initialImageWidth * pixelDensityIncrease)
      extensionHeight = int(initialImageHeight * pixelDensityIncrease) 

    # print(extensionHeight)

    # next create variables for extending / adding transparency to the images:
    color =  (0, 0, 0, 0) # the last number, 0 means the value of the extension will be completely transparent if we save as .png
    extendTop = 0 # we start at zero since we start from the top left corner
    extendBottom = int((extensionHeight - imageWidth) + imageWidth) # we start with a value since there is space on the bottom in the 1st image; example 2560
    extendLeft = 0 # we start at zero since we start from the top left corner
    extendLeft = int((imageWidth / 2) - imageWidth) # we actually start in the negative since we begin by adding + 256 to the left, which brings us to 0 initially
    extendRight = int(extensionWidth +  (imageWidth/2)) # we start with a value since there is space on the right in the 1st image; example 2304, then subtract 256

    # we have to check which image we're on, both in total, by column and by row:
    image_counter = 0
    countW = 1
    countH = 0

    # these values controls how the mask looks and we also need to gather the resulting img + mask for when we stitch them back together:
    maskBorderAmount = 96
    maskBlurIntensity = 16
    listOfExportedImages = []
    listOFExportedImageMasks = []

    # before processing the images, check if a folder to store the new images exists:
    # filePathSave = directory + "\\" + "extendedFrame" + "\\"
    filePathSave = "/content/HD/extendedTiles/"
    isExist = os.path.exists(filePathSave)
    if not isExist:
        os.makedirs(filePathSave)    

    # now cycle through the images and extend them, etc:
    for image in listOfImages:
        # read and number the image:
        image_counter += 1
        image_counterPadded = str(image_counter).zfill(4) # the number we label each image is padded; example 0001
        img = Image.open(directory + '/' + image) # load the image
        img = img.convert("RGBA") # make sure this image supports transparency

        # calculate how much to extend the canvas in all directions:
        if countW <= columnsNumber: # if the count is lower than the number of images width wise, we only need to extend on the width 
            countW += 1
            extendLeft += int(imageWidth / 2) # example: 256
            extendRight -= int(imageWidth / 2)
            # print("exension amount left: " + str(extendLeft))
        elif countW > columnsNumber:
            # once we exceed the number of images width wise, we reset the count/width and change the spacing just once for the height
            countW = 2
            countH +=1
            extendTop += int(imageWidth / 2) # example: 256
            extendBottom -= int(imageWidth / 2)
            extendLeft = 0
            extendRight = extensionWidth

        # finally calculate how to edit the dimensions of the transparent bits of the image:
        # width, height = img.size
        new_width = extendRight + extendLeft
        new_height = extendTop + extendBottom
        result = Image.new(img.mode, (new_width, new_height), color)
        result.paste(img, (extendLeft, extendTop))
        
        # count which actual column and row we're on:
        image_rowPadded = str(countH+1).zfill(3)
        image_columnPadded = str(countW-1).zfill(3)

        # create a feather mask around the cropped image to blend it in better when compositing:
        mask_im = Image.new("L", result.size, 0)
        draw = ImageDraw.Draw(mask_im)
        # draw.rectangle((extendLeft+ 12, extendTop + 12, extendRight - (extendRight-imageWidth) -18, extendBottom - (extendBottom - imageWidth)-18), fill=255)
        #maskX = int((imageWidth/2) * (countW - 2) + maskBorderAmount)
        #maskY = int((imageWidth/2) * (countH) + maskBorderAmount)
        if countW == 2:
            maskX = int((imageWidth/2) * (countW - 2))
        else:
            maskX = int((imageWidth/2) * (countW - 2) + maskBorderAmount)
        if countH == 0:
            maskY = int((imageWidth/2) * (countH))
        else:
            maskY = int((imageWidth/2) * (countH) + maskBorderAmount)
        maskW = int(((imageWidth/2) * (countW - 2)) + imageWidth - maskBorderAmount)
        maskH = int(((imageWidth/2) * (countH)) + imageWidth - maskBorderAmount)
        draw.rectangle((maskX, maskY, maskW, maskH), fill=255) # THIS is where we draw the MASK, fill=255 must mean it's a perfect white? I forgot
        # create a blur for the mask:
        mask_im_blur = mask_im.filter(ImageFilter.GaussianBlur(maskBlurIntensity)) # maskBlurIntensity is the variable we define to control how intense to mask

        # finally, save this image in a new folder for the extended canvas version:
        # filePathSaveFull = filePathSave + "paddedImage" + "_" + image_counterPadded + ".png"
        filePathSaveFull = filePathSave + "paddedImage" + "_" + image_rowPadded + "_" + image_columnPadded + ".png"
        result.save(filePathSaveFull, quality=100)
        listOfExportedImages.append(result)
        # save the mask too (actually I rather not save the mask, these are easy to make in Photoshop and this saves time not saving):
        # filePathSaveFull = filePathSave + "mask" + "_" + image_counterPadded + ".png"
        # mask_im_blur.save(filePathSaveFull, quality=100)
        listOFExportedImageMasks.append(mask_im_blur) # instead of saving the masks, save them to a variable
        output.clear()
        print("Extended: " + str(image_counter) + " / " + str(len(listOfImages)))

    # after extending the frames and creating the masks, combine them into a single image:
    back_img = listOfExportedImages[0].copy()
    countMask = 0
    filePathSaveFull = "/content/HD/" + "_final_HD" + ".png"

    for image in listOfExportedImages:
        #listOfExportedImages = []
        #listOFExportedImageMasks = []
        if image == listOfExportedImages[0]:
            back_img.save(filePathSaveFull, quality=100)
        else:
            #back_img = Image.open(filePathSaveFull) # load the image
            back_img.paste(image, (0,0), listOFExportedImageMasks[countMask])
            #back_img.save(filePathSaveFull, quality=100)
        countMask += 1
        output.clear()
        print("Processed: " + str(countMask) + " / " + str(len(listOfExportedImages)))
    back_img.save(filePathSaveFull, quality=100)
    print("done")

# img_new = add_margin(r"H:\New\testUpres\testCodeHD", 512, 8)
img_new = add_margin(r"/content/HD/postProcessedGridSplits", numberOfImageAlongX) # you need to include input folder, make sure only the # of images you want are in there and include # of total images in width



# --------------------------------------------------------------------

# Now we copy the final files to Google Drive to back them up:
print("Wait while copying to Google Drive...")
import shutil
copyAddressOnDrive = "/content/gdrive/MyDrive/AI/StableDiffusionHD/" + batch_name

directoryToCopy = "/content/HD/extendedTiles/"
listOfImagesToCopy = [f for f in listdir(directoryToCopy) if f.endswith(".png") and isfile(join(directoryToCopy, f))]
listOfImagesToCopy.sort()

isExist = os.path.exists(copyAddressOnDrive + "/tiles/")
if not isExist:
    os.makedirs(copyAddressOnDrive + "/tiles/")

for x in listOfImagesToCopy:
  fullPath = directoryToCopy + x
  shutil.copy(fullPath, copyAddressOnDrive + "/tiles/")

shutil.copy("/content/HD/_final_HD.png", copyAddressOnDrive)
print("Done copying files. Enjoy!")

## <u> B) Post Processing - **Face Recognition Upscaling** </u> 

In [ ]:
# Put each of your prompts on a new line:
# NOTE: you must run this cell once at start up and whenever you make an edit!
promptsListForOpenCV = '''
blue skies
female magician
attractive American Woman
'''

In [ ]:
#@title Face Recognition & Upscale (Run):
#@markdown `This script will process all the images in the imgPath location, sequentially by name`<br>
#@markdown `Turn on useRealESRGANforUpscale for higher quality results, just takes longer`<br>
#@markdown `If it fails, the ESRGAN scale may be too high, your input image is too high res or the grid ratio is too high`<br>
#@markdown `Predictable low end results: input images size 640x640, scale factor 2, grid ratio 672, init_strength 0.5`<br>

##@markdown

import re
import random
# facial recognition code:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from google.colab import files
from base64 import b64decode
from PIL import Image, ImageDraw, ImageFilter
import numpy as np
import os
import sys
import cv2
import glob
import shutil
from os import listdir
from os.path import isfile, join
import time



# Create the folders & folder variables we need: -----------
ProjectName = "testFaceRecognitionC" #@param {type:"string"}
# the folder to contain all the other folders:
FRpathBase = "/content/facialRecognition/"
os.makedirs(FRpathBase, exist_ok=True) 
# made a project for everything as well, why not:
FRpathProject = "/content/facialRecognition/" + ProjectName + "/"
os.makedirs(FRpathProject, exist_ok=True) 
# where the study material (ex previewing how the crops will look) are saved
FRpathProjectWIP = "/content/facialRecognition/" + ProjectName + "/workFiles/"
os.makedirs(FRpathProjectWIP, exist_ok=True) 
# where the pre-gen crops will be saved:
FRpathProjectCrops = "/content/facialRecognition/" + ProjectName + "/crops/"
os.makedirs(FRpathProjectCrops, exist_ok=True) 
# where the newly generated but still cropped faces will be saved:
FRpathProjectUprezzedCrops = "/content/facialRecognition/" + ProjectName + "/uprezzedCrops/"
os.makedirs(FRpathProjectCrops, exist_ok=True) 
# where the final upscaled and face swapped images will be saved:
FRpathProjectFinal = "/content/facialRecognition/" + ProjectName + "/finals/"
os.makedirs(FRpathProjectFinal, exist_ok=True) 
# where the real-ESRGAN upscaled images will be temp saved:
FRpathProjectUpscaledINIT = "/content/facialRecognition/" + ProjectName + "/upscaledINIT/"
os.makedirs(FRpathProjectUpscaledINIT, exist_ok=True) 
# where the extended transparent results are stored:
FRpathProjectExtended = "/content/facialRecognition/" + ProjectName + "/extended/"
os.makedirs(FRpathProjectExtended, exist_ok=True) 

# clean up the work files (only clean up the base directory):
outputDirectoryProcessed = FRpathBase
outputDirectoryFiles = glob.glob(FRpathBase + '/**/*', recursive=True)
if os.path.exists(outputDirectoryProcessed) == False:
  os.makedirs(outputDirectoryProcessed)
# clean up existing files in the output Directory:
for f in outputDirectoryFiles:
    try:
        os.remove(f)
    except OSError as e:
        pass

# save the positions to lists to call later:
countOfFacesPerInput = []
positionFaceImageXList = []
positionFaceImageYList = []
positionFaceImageWList = []
positionFaceImageHList = []
expandFacialRecognitionRatio = 15 

# A) Check the Image for Faces and crop it: -------------------------------------------
imgPath = "/content/init_image/" #@param {type:"string"}
imgPathInput = [f for f in listdir(imgPath) if isfile(join(imgPath, f))]
imgPathInput.sort # this variable gives a list of all the images in the init_image folder (ex. [img1.png, img2.png])
numberOfImagesToProcess = len(imgPathInput)
counter = 0
useRealESRGANforUpscale = True #@param {type:"boolean"}
ESRGANscaleFactor = 3 #@param {type:"slider", min:2, max:4, step:0.5}
#@markdown `for faceRecognitionMinimizer, higher values = less faces detected. Default is 3-5`<br>
faceRecognitionMinimizer = 3 #@param {type:"slider", min:1, max:10, step:1}

# ------------------------------------------------------
# Real-ESRGAN Upscaling for the Input Images:
# since it takes a while to upscale, make this feature optional:
if useRealESRGANforUpscale == True:
  # it seems only png work, so convert .jpg to PNG:
  for img in imgPathInput:
    if img.endswith('.jpg'):
      im1 = Image.open(imgPath + img)
      splitPath = os.path.splitext(imgPath + img)[0]
      newPath = splitPath + '.png'
      im1.save(newPath)
      os.remove(imgPath + img)
  imgPathInput = [f for f in listdir(imgPath) if isfile(join(imgPath, f))]
  imgPathInput.sort # this variable gives a list of all the images in the init_image folder (ex. [img1.png, img2.png])

  #1) set up the file to load into the ESRGAN:
  # note: these folders live inside the real ESRGAN created folders, which is why they do not have /content/ in their name
  upload_folder = 'upload' # wierd directories that ESRGAN uses
  result_folder = 'results'# ignore the lack of /content/ directory

  # create the upload and results folders + clean them up:
  if os.path.isdir(upload_folder):
      shutil.rmtree(upload_folder)
  if os.path.isdir(result_folder):
      shutil.rmtree(result_folder)
  os.mkdir(upload_folder)
  os.mkdir(result_folder)

  # upload images: (in this case we read our init_images)
  uploadPath = imgPath
  filesList = imgPathInput
  for filename in filesList:
    dst_path = os.path.join(upload_folder, filename)
    print(f'copy {filename} to {dst_path}')
    shutil.copy(uploadPath + filename, dst_path)

  # 2) Core Code to Make ESRGAN Uprez the Image:
  # if it is out of memory, try to use the `--tile` option
  # We upsample the image with the scale factor X3.5
  if ESRGANscaleFactor == 2:
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2 --face_enhance
  elif ESRGANscaleFactor == 2.5:
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2.5 --face_enhance  
  elif ESRGANscaleFactor == 3:
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 3 --face_enhance
  elif ESRGANscaleFactor == 3.5:
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 3.5 --face_enhance
  elif ESRGANscaleFactor == 4:
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 4 --face_enhance
  # Arguments
  # -n, --model_name: Model names
  # -i, --input: input folder or image
  # --outscale: Output scale, can be arbitrary scale factore. 
  input_folder = '/content/stable-diffusion/Real-ESRGAN/upload'
  result_folder = '/content/stable-diffusion/Real-ESRGAN/results'
  input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
  output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
  output_list.sort()
  #print(input_list)
  #print(output_list) # list of upressed files

  # now we replace the original variables with our new images:
  imgPath = result_folder + "/"
  imgPathInput = [f for f in listdir(imgPath) if isfile(join(imgPath, f))]
  imgPathInput.sort # this variable gives a list of all the images in the init_image folder (ex. [img1.png, img2.png])
  time.sleep(1)
  # copy the files to easer to see location:
  for item in imgPathInput:
    fullPath = imgPath + item
    shutil.copy(fullPath, FRpathProjectUpscaledINIT)

  imgPath = FRpathProjectUpscaledINIT
  imgPathInput = [f for f in listdir(imgPath) if isfile(join(imgPath, f))]
  imgPathInput.sort # this variable gives a list of all the images in the init_image folder (ex. [img1.png, img2.png])
# print("imgPathInput:" + str(imgPathInput))
  # ------- END real ESRGAN -----------------



checkIfRealFace = 0
# -------------------------------------------------------------------------------
# -------------------------FACIAL RECOGNITION: ----------------------------------
# A) Check the Image for Faces and crop it: -------------------------------------------
for image in imgPathInput:
  counter +=1
  counterString = str(counter)
  inputImageFullPath = imgPath + image
  imgPathOutput = FRpathProjectWIP + "study_" + counterString.zfill(4) + ".png" # ex. study1.png
  # print(imgPathInput)
  # we need to save the x and y coordinates of the face image so we can swap it out later:
  # we also need to save the length and width:

  positionFaceImageX = 0
  positionFaceImageY = 0
  positionFaceImageW = 0
  positionFaceImageH = 0

  # 1) read the image from path:
  img = cv2.imread(inputImageFullPath)
  imgCopy = img.copy() # we create a copy of the input image to purform the cropping on since we use the original link to create the study images
  fullImageHeight, fullImageWidth, channels  = img.shape # we need to know the full image width to know the ratio to expand the recognition

  # 2) convert image to grayscale (required by image recognition model):
  gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  # 3) Use the Haar Cascade model to detect faces from the image:
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
  # it seems higher resolution images requires us to modify the intensity of searching for faces, which is this value:
  if useRealESRGANforUpscale == False:
    faces = face_cascade.detectMultiScale(gray_img,1.1,faceRecognitionMultiplier)
  elif useRealESRGANforUpscale == True:
    faces = face_cascade.detectMultiScale(gray_img,1.1,faceRecognitionMultiplier*2)

  # 4) print the number of faces detected:
  #print(f"len(faces)} faces detected in the image.")
  numberOfFacesDetected = len(faces)
  counterFaces = 0
  countOfFacesPerInput.append(numberOfFacesDetected)
  print("Number of faces detected:" + str(numberOfFacesDetected))

  # 5a) Draw rectangle around the faces and crop the faces:
  # 5b) also save the correct position data (the distance of top left corner + crop W & H):
  for f in faces:
    counterFaces +=1
    counterFacesString = str(counterFaces)
    x1, y1, w1, h1 = [ v for v in f ]
    calculateDistancePixelsPercentage = int(fullImageWidth / expandFacialRecognitionRatio)
    calculateDistancePixels = int(calculateDistancePixelsPercentage)
    positionFaceImageX = x = int(x1 - calculateDistancePixels)
    positionFaceImageY = y = int(y1 - calculateDistancePixels)
    positionFaceImageW = w = int(w1 + (calculateDistancePixels * 2))
    positionFaceImageH = h = int(h1 + (calculateDistancePixels * 2))
    positionFaceImageXList.append(positionFaceImageX)
    positionFaceImageYList.append(positionFaceImageY)
    positionFaceImageWList.append(positionFaceImageW)
    positionFaceImageHList.append(positionFaceImageH)

    sub_face = imgCopy[y:y+h, x:x+w]

    fname, ext = os.path.splitext(inputImageFullPath)
    # cv2.rectangle(img, (x,y), (x+w,y+h), (256,256,256), thickness=20) # FOR DEBUGGING IF THE IMAGE IS BEING PLACED CORRECTLY BACK ON THE FULL IMG
    #cv2.imwrite("/content/known/" + fname+"_cropped_"+ext, sub_face)
    #cv2.imwrite(FRpathProjectCrops + "cropped" + ".png", sub_face)
    try:
      cv2.imwrite(FRpathProjectCrops + "cropped_" + counterString.zfill(4) + "_f" + counterFacesString.zfill(2) + ".png", sub_face)
      cv2.rectangle(img, (x,y), (x+w,y+h), (256,256,256), thickness=2)
      cv2.rectangle(img, (x1, y1), (x1 + w1, y1+h1), color=(0, 256, 0), thickness=1) 
      #break # only process the first face result, if there are multiple faces those get skipped
      checkIfRealFace += 1
    except:
      "not a face!"

  # save the image with rectangles
  cv2.imwrite(imgPathOutput, img) # the study image

# since we may have more than one face in the images, calculate the total # of images overall:
numberOfFacesDetected = checkIfRealFace

totalNumberOFCrops = 0
for x in countOfFacesPerInput:
  totalNumberOFCrops += x

# ----------------------END FACIAL RECOGNITION P1 -------------------------------
# -------------------------------------------------------------------------------






# -------------------------------------------------------------------------------
# B) Run Stable Diffusion to Process Newly Generated Image:


#B1) first deal with getting the input images and the prompts:
promptList = []
promptList = ["an attract female portrait by artgerm, greg rutkowski, trending on artstation, detailed, rendered in Octane, ethereal"]

# create a list of all the prompts from new line:
if type(promptsListForOpenCV) is str:
  promptsListForOpenCV = [i for i in promptsListForOpenCV.split('\n') if i]

# -----------------------------------------------------------------------
# 3) Now we can run these prompts and images through Stable Diffusion to Upres them!


#@markdown Basic settings:
batch_name = ProjectName
gridRatio = "512" #@param [512, 608, 640, 672, 704]
width_height = [int(gridRatio), int(gridRatio)] #param{type: 'raw'}
guidance_scale = 25 #@param {type:"number"}
steps = 7 #@param {type:"integer"}
init_strength = 0.4 #@param {type:"number"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
num_batch_images = 1 #param {type:"integer"}
sampler = 'ddim' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 #param {type:"number"}
seed = -1 #@param {type:"integer"}

# --------------------------------------------------------------------------------
# now get the input images from the folder we created earlier:

# find all the images in the folder meant to read them:
numberOfImages = 0
init_imgs_list = []
for path in os.listdir(FRpathProjectCrops): # note: imgPath is defined as a parameter at the beginning of the code
    if os.path.isfile(os.path.join(FRpathProjectCrops, path)):
        numberOfImages += 1
        init_imgs_list.append(path)
init_imgs_list.sort()
#print(numberOfImages)
#print(init_imgs_list)

# --------------- Next we deal with Prompts: -------------
#@markdown Prompt
promptStart = "female" #@param {type:"string"}
promptEnd = "portrait, young woman, detailed gorgeous face, digital art, painting, artstation, concept art, smooth, sharp focus, high definition, detailed, illustration, art by artgerm and greg rutkowski and alphonse mucha" #@param {type:"string"}
promptList = []
promptListFullString = []

# append the text strings to an array to combine with full prompt later:
for x in promptsListForOpenCV:
  # this one will just have the prompts we have written + the prompt template, it will not duplicate prompts over multi face images yet
  promptListFullString.append(promptStart + ", " + x + ", " + promptEnd)

# check if we actually wrote enough lines in our text file, if not we'll add empty strings:
if len(promptListFullString) != numberOfImagesToProcess: # number of images to process is the total number of INIT images
  extraLinesNeeded = numberOfImagesToProcess - len(promptListFullString)
  for x in range(extraLinesNeeded):
    promptListFullString.append(promptStart + ", " + promptEnd)

counter = 0
# now we check how many of these lines we need to duplicate for multi face images:
for x in countOfFacesPerInput: # this tells us how many faces are in per image; ex [0, 1, 6]
  if x == 0:
    # if there are no faces in the image, skip appending that line
    counter += 1
  else:
    for y in range (x):
      promptList.append(promptListFullString[counter]) # append this line as many times as there are faces in the image
    counter += 1 # after appending all the neccessary lines, move on to the next line

#print(len(promptList))
#for x in promptList:
#  print(x)
# ------ NOW WE HAVE OUR FULL TEXT FOR THE PROMPTS! ----------------------------
#print(len(init_imgs_list))
#print(init_imgs_list)



# set up counters and the output folder:
batch_idx = 0
sample_idx = 0
# numberPosition = True
# opt.saveNumberPosition == numberPosition


# --------------------------------------------------------------------------------
# now get to prepare the images, prompts and settings for Stable Diffusion:
for num in range((totalNumberOFCrops)):
  print("Processing image: " + (str(num+1)) + " / " + str(totalNumberOFCrops))
  if len(init_imgs_list) == 0:
    print("No faces detected! Exiting")
    break
  opt.init_img = (FRpathProjectCrops + str(init_imgs_list[int(num)]))
  opt.ddim_steps = steps
  opt.n_iter = 1
  opt.n_samples = samples_per_batch
  # opt.outdir = os.path.join(outputs_path, batch_name)
  opt.outdir = FRpathProjectUprezzedCrops
  opt.prompt = promptList[int(num)]
  opt.sampler = sampler
  opt.scale = guidance_scale
  opt.seed = random.randint(0, 2**32) if seed == -1 else seed
  opt.strength = max(0.0, min(1.0, 1.0 - init_strength))
  opt.W, opt.H = map(lambda x: x - x % 64, (width_height[0], width_height[1])) # resize to integer multiple of 64

  if opt.init_img != None and opt.init_img != '':
      opt.sampler = 'ddim'

  if opt.sampler != 'ddim':
      opt.ddim_eta = 0.0

  # save settings
  settings = {
      'ddim_eta': ddim_eta,
      'guidance_scale': guidance_scale,
      'init_image': init_imgs_list[num],
      'init_strength': init_strength,
      'prompt': promptList[num],
      'sampler': sampler,
      'samples_per_batch': samples_per_batch,
      'seed': opt.seed,
      'steps': steps,
      'width': opt.W,
      'height': opt.H,
  }
  os.makedirs(opt.outdir, exist_ok=True)
  #with open(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt", "w+", encoding="utf-8") as f:
  #    json.dump(settings, f, ensure_ascii=False, indent=4)
  sample_idx = 0

  gc.collect()
  torch.cuda.empty_cache()

  images = generate(opt)

  clear_output(wait=True)
  print(f"Used seed: {opt.seed}")
  print(f"Saved to: {opt.outdir}")
  print("Processed image: " + str((num+1)) + " / " + str(totalNumberOFCrops))
  display(images[0])

  batch_idx += 1
  sample_idx += 1

# ------- END OF STABLE DIFFUSION PROCESSING ------------------

# Clean up the files in /uprezzedCrops/
# now some clean up before merging the cropped face images into the original images:

# FRpathProjectFinal # /content/facialRecognition/testFacialRecognition/finals

directoryToRename = FRpathProjectUprezzedCrops # /content/facialRecognition/testFacialRecognition/uprezzedCrops
# rename the images to clean them up:
listOfImagesToRename = [f for f in listdir(directoryToRename) if f.endswith(".png") and isfile(join(directoryToRename, f))]
listOfImagesToRename.sort()
counter = 1
listOfImagesToIntegrate = []
for x in listOfImagesToRename:
  counterString = str(counter)
  # newName = "result_" + counterString.zfill(4) + ".png"
  newName = "result_" + counterString.zfill(4) + ".png"
  src =f"{directoryToRename}/{x}" # the incorrect file names we need to rename
  dst =f"{directoryToRename}/{newName}"
  os.rename(src, dst)
  counter += 1

listOfImagesToIntegrate = [f for f in listdir(directoryToRename) if f.endswith(".png") and isfile(join(directoryToRename, f))]
listOfImagesToIntegrate.sort()
# end clean up


# ------- RESIZING AND STITCHING CODE TO FINISH IT OFF: ---------------
counter = 0
selectInitialImage = 0
countRepeatPrompts = 0
tempCollectExtendedImg = []
imgPathOutputFin = ""
for image in listOfImagesToIntegrate:
  # RESIZE IMAGES: -------------------------------------------
  # since we generate the close up crops in a larger resolution, we have to unify them with the original images

  # first gather information from the images in our folders:
  imgSizeCropped = Image.open(FRpathProjectCrops + '/' + init_imgs_list[counter]) # load the image from /crops/
  w, h = imgSizeCropped.size # shoud be on the small side
  imageWidthCropped = w

  imgSizeGenerated = Image.open(FRpathProjectUprezzedCrops + '/' + listOfImagesToIntegrate[counter]) # load the image from /uprezzedCrops/
  w, h = imgSizeGenerated.size # should be 512 pixels +
  imageWidthGenerated = w # could be 512, 640, etc, it reads the actual image width so we don't guess

  pixelDensityIncrease = imageWidthGenerated / imageWidthCropped # could be 1.25 which is 25% larger
 
  if useRealESRGANforUpscale == False:
    # easy way to unify: downscale the upscaled crop to fit the original crop:
    imagePathFull = Image.open(FRpathProjectUprezzedCrops + image)
    new_image = imagePathFull.resize((imageWidthCropped, imageWidthCropped))
  elif useRealESRGANforUpscale == True:
    # harder way to unify; upscale
    # although since we used this variable to swap the original input images for the upscaled ones, it might be easier than I think
    imagePathFull = Image.open(FRpathProjectUprezzedCrops + image)
    new_image = imagePathFull.resize((imageWidthCropped, imageWidthCropped))

  #print(image)
  #print(imgPathInput)
  #print(imgPath + imgPathInput[counter])
  # -------------------------------------------------------------------------------
  # -------------------------FACE RECOGNITION: ----------------------------------
  # D) merge cropped image back to the original: ---------------------------------------------------

  # this code will take 2 images (1 full and 1 crop) and paste the cropped image at proper coordinates
  #Read the two images:
  # image1 = Image.open('/content/known/y1.png')
  # image1 = Image.open(imgPath + imgPathInput[counter])
  # to get image1 which is the full image which may not correspond to the crop #, we have to determine which initial images should go with this crop:
  currentInitialImageWork = countOfFacesPerInput[selectInitialImage] # example: [0, 1, 6]
  # select which initial image we're on:
  while currentInitialImageWork == 0:
    if currentInitialImageWork == 0:
      # this image has no faces, so we should not process on this initial image
      selectInitialImage += 1
      currentInitialImageWork = countOfFacesPerInput[selectInitialImage] # example: [0, 1, 6]
    if currentInitialImageWork != 0: # if this image has a face, we can stop looking for an image with a face
      break
  image1 = Image.open(imgPath + imgPathInput[selectInitialImage]) # the initial base image
  countRepeatPrompts += 1
  image2 = new_image # the cropped image
  # image2 = image2.convert("RGBA") # make sure the image supports transparency
  # now we have the images to process with ----

  # create a feather mask around the cropped image to blend it in better:
  mask_im = Image.new("L", image2.size, 0)
  draw = ImageDraw.Draw(mask_im)
  x, y = image2.size # height and width of the crop image
  distance = y / 15 # framing of mask that will contain the blur region
  draw.rectangle((distance, distance, positionFaceImageWList[counter]-distance, positionFaceImageHList[counter]-distance), fill=255)
  # create a blur for the mask:
  mask_im_blur = mask_im.filter(ImageFilter.GaussianBlur(4))
  # mask_im_blur.save('/content/mask_rectangle.png')

  counterString = str(counter+1)
  # now we output this image:
  new_image1 = image1.copy()
  x, y = new_image1.size # height and width of the base image
  new_imageTransparent = Image.new("RGBA",(x,y),(0,0,0,0)).convert("RGBA") # full transparency of the input image
  new_imageBlack = Image.new("RGBA",(x,y),(0,0,0)) # fully black of the input image
  new_imageTransparentBlend = new_imageTransparent.copy()

  stringImageNumber = str(selectInitialImage+1)
  stringcountRepeatPrompts = str(countRepeatPrompts)
  imgPathOutput = FRpathProjectExtended + "extended_" + stringImageNumber.zfill(4) + "_f" + stringcountRepeatPrompts.zfill(2) + ".png" # ex. finished_0001.png
  # note: the position X and Y variables must be calculated earlier!

  new_image = image1.copy()
  new_image.paste(image2, (positionFaceImageXList[counter],positionFaceImageYList[counter]), mask_im_blur)
  new_imageTransparent.paste(image2, (positionFaceImageXList[counter],positionFaceImageYList[counter]))
    
  #new_image.save(imgPathOutput) # new_image is the full image
  new_imageTransparent.save(imgPathOutput) # the transparent cropped images

  tempCollectExtendedImg.append(new_imageTransparent) # add this new extended crop to an array so we can apply it to the full image after processing it
  counter +=1

  # counter for the base image selection / image1:
  # basically this section runs when we have finished processing a base image and ready to merge the generation crops finally with the base!
  if countRepeatPrompts == countOfFacesPerInput[selectInitialImage]: # once we're run through all the crops for this intiial image, we have to move to the next one
    selectInitialImage += 1
    newMerge = new_image.copy() # temp save for image to be pasted on
    # at this point we also create the final result by merging the collected extended images with the input image (image1):
    imgPathOutputFin = FRpathProjectFinal + "final_" + stringImageNumber.zfill(4) + "_f" + stringcountRepeatPrompts.zfill(2) + ".png" # ex. finished_0001.png

    for extendedImg in tempCollectExtendedImg:
      newMerge.paste(extendedImg,(0,0), extendedImg)
      newMerge = newMerge.copy()
      newMerge.save(imgPathOutputFin)
    countRepeatPrompts = 0
    tempCollectExtendedImg.clear() # get ready for appending the next round of crops
  # continue until for loop is done...

# ----------------------END FACIAL RECOGNITION P2 -------------------------------
# -------------------------------------------------------------------------------



# --------------------------------------------------------------------
# --------------------------------------------------------------------

# Now we copy the final files to Google Drive to back them up:
if len(init_imgs_list) != 0:
  print("Wait while copying to Google Drive...")
  copyAddressOnDrive = "/content/gdrive/MyDrive/AI/StableDiffusionUpscaleFaces/" + ProjectName

  directoryToCopy1 = FRpathProjectFinal # final images
  directoryToCopy2 = FRpathProjectUpscaledINIT # upscaled INIT images
  directoryToCopy3 = FRpathProjectExtended # extended crops for photobashing
  listOfImagesToCopy1 = [f for f in listdir(directoryToCopy1) if f.endswith(".png") and isfile(join(directoryToCopy1, f))]
  listOfImagesToCopy1.sort()
  listOfImagesToCopy2 = [f for f in listdir(directoryToCopy2) if f.endswith(".png") and isfile(join(directoryToCopy2, f))]
  listOfImagesToCopy2.sort()
  listOfImagesToCopy3 = [f for f in listdir(directoryToCopy3) if f.endswith(".png") and isfile(join(directoryToCopy3, f))]
  listOfImagesToCopy3.sort()

  for index, file in enumerate(listOfImagesToCopy2):
    valueSt = str(index+1) 
    newName = "upscaledInput_" + valueSt.zfill(4)
    os.rename(os.path.join(directoryToCopy2, file), os.path.join(directoryToCopy2, ''.join([newName, '.png'])))  
  listOfImagesToCopy2 = [f for f in listdir(directoryToCopy2) if f.endswith(".png") and isfile(join(directoryToCopy2, f))]
  listOfImagesToCopy2.sort()

  isExist = os.path.exists(copyAddressOnDrive + "/001")
  if not isExist:
      os.makedirs(copyAddressOnDrive + "/001")
      copyAddressOnDrive = copyAddressOnDrive + "/001"
  else:
    i=1
    while os.path.exists(copyAddressOnDrive + "/" + str(i).zfill(3)):
      i+=1
    os.makedirs(copyAddressOnDrive + "/" + str(i).zfill(3))
    copyAddressOnDrive = copyAddressOnDrive + "/" + str(i).zfill(3)

  for x in listOfImagesToCopy1:
    fullPath = directoryToCopy1 + x
    shutil.copy(fullPath, copyAddressOnDrive)

  for x in listOfImagesToCopy2:
    fullPath = directoryToCopy2 + x
    shutil.copy(fullPath, copyAddressOnDrive)

  for x in listOfImagesToCopy3:
    fullPath = directoryToCopy3 + x
    shutil.copy(fullPath, copyAddressOnDrive)

  print("Please wait for files to copy to Google Drive. Enjoy!")

In [ ]:
#@title Experimental Gender & Age Detection (study only):
#@markdown `note: this script will not do more than study an image and guess the gender/age`<br>
#@markdown `Gender detection is fine, but the age detection is way off`<br>
#@markdown https://data-flair.training/blogs/image-segmentation-machine-learning/


# models are required to be added manually to /content/gad/
# models are from here: https://data-flair.training/blogs/python-project-gender-age-detection/
# this code was written and taken from here: https://dev.to/ethand91/simple-age-and-gender-detection-using-python-and-opencv-319h


# Gender and Age Detection Code Fragment

import cv2
import math
import sys
from google.colab.patches import cv2_imshow

# Defined the model files
pathStart = "/content/gad/"
FACE_PROTO = pathStart + "opencv_face_detector.pbtxt"
FACE_MODEL = pathStart + "opencv_face_detector_uint8.pb"

AGE_PROTO = pathStart + "age_deploy.prototxt"
AGE_MODEL = pathStart + "age_net.caffemodel"

GENDER_PROTO = pathStart + "gender_deploy.prototxt"
GENDER_MODEL = pathStart + "gender_net.caffemodel"

# Load network
FACE_NET = cv2.dnn.readNet(FACE_MODEL, FACE_PROTO)
AGE_NET = cv2.dnn.readNet(AGE_MODEL, AGE_PROTO)
GENDER_NET = cv2.dnn.readNet(GENDER_MODEL, GENDER_PROTO)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
AGE_LIST = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
GENDER_LIST = ["Male", "Female"]

box_padding = 20

def get_face_box (net, frame, conf_threshold = 0.7):
  frame_copy = frame.copy()
  frame_height = frame_copy.shape[0]
  frame_width = frame_copy.shape[1]
  blob = cv2.dnn.blobFromImage(frame_copy, 1.0, (300, 300), [104, 117, 123], True, False)

  net.setInput(blob)
  detections = net.forward()
  boxes = []

  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > conf_threshold:
      x1 = int(detections[0, 0, i, 3] * frame_width)
      y1 = int(detections[0, 0, i, 4] * frame_height)
      x2 = int(detections[0, 0, i, 5] * frame_width)
      y2 = int(detections[0, 0, i, 6] * frame_height)
      boxes.append([x1, y1, x2, y2])
      cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height / 150)), 8)

  return frame_copy, boxes

def age_gender_detector (input_path):
  image = cv2.imread(input_path)
  frame = image.copy()
  frame_face, boxes = get_face_box(FACE_NET, frame)

  for box in boxes:
    face = frame[max(0, box[1] - box_padding):min(box[3] + box_padding, frame.shape[0] - 1), \
      max(0, box[0] - box_padding):min(box[2] + box_padding, frame.shape[1] - 1)]

    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB = False)
    GENDER_NET.setInput(blob)
    gender_predictions = GENDER_NET.forward()
    gender = GENDER_LIST[gender_predictions[0].argmax()]
    print("Gender: {}, conf: {:.3f}".format(gender, gender_predictions[0].max()))

    AGE_NET.setInput(blob)
    age_predictions = AGE_NET.forward()
    age = AGE_LIST[age_predictions[0].argmax()]
    print("Age: {}, conf: {:.3f}".format(age, age_predictions[0].max()))

    label = "{},{}".format(gender, age)
    cv2.putText(frame_face, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

  return frame_face


if __name__ == "__main__":
  inputImageFullPath = "/content/init_image/a0011.png" #@param {type:"string"}
  output = age_gender_detector(inputImageFullPath)
  cv2.imwrite("output.jpg", output)
  cv2_imshow(output)

## C) Upscale from File Browser (Real ESRGAN)

In [ ]:
import cv2
import os
import glob
from google.colab import files
import shutil
from os import listdir
from os.path import isfile, join

#@markdown `note: run this to upres from selecting file browser`<br>
#1) set up the file to load into the ESRGAN:
upload_folder = 'upload' # wierd directories that ESRGAN uses
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
# upload = "/content/1.jpg" #@param {type:"string"}
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.copy(filename, dst_path)

# ---------------------------------------------------------------------------------------------
# 2) Core Code to Make ESRGAN Uprez the Image:
# if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2 --face_enhance
# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore. 
# ---------------------------------------------------------------------------------------------

input_folder = '/content/stable-diffusion/Real-ESRGAN/upload'
result_folder = '/content/stable-diffusion/Real-ESRGAN/results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
output_list.sort()
print(input_list)
print(output_list)

In [ ]:
#@markdown `Download the Upscaled Images as .zip`<br>

# Make a zip file of your results to download:
from google.colab import files
import os
file_path = "/content/stable-diffusion/Real-ESRGAN/results.zip"
if os.path.isfile(file_path):
  os.remove(file_path)
!zip -r results.zip /content/stable-diffusion/Real-ESRGAN/results
files.download('results.zip')

## <u>Utility:</u> Clear Image Folder

In [ ]:
import os
import glob

files = glob.glob('/content/init_image/**/*', recursive=True)

for f in files:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
Info = 'run to clear image folder' #@param ["run to clear image folder"]